## INFO442
## News Article Recommender System
## Group 3 (Kevin Shi, Benjamin Leung, Kathryn Swatek, Richardson Chhin)
## Notebook for

## Any Notes:
...

## Imports

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

## Preprocessing for validation set

### Load behaviors_train_preprocessed.parquet

In [ ]:
# REMINDER: This is use to load the data back to the original format from parquet files
try:
    behaviors_train_preprocessed = pd.read_parquet('MINDsmall_train_processed/behaviors_train_preprocessed.parquet')
except FileNotFoundError:
    from google.colab import files
    files.upload()
    behaviors_train_preprocessed = pd.read_parquet('behaviors_train_preprocessed.parquet')

Saving behaviors_train_preprocessed.parquet to behaviors_train_preprocessed (1).parquet


In [ ]:
print(type(behaviors_train_preprocessed))
print(behaviors_train_preprocessed.shape)
print(behaviors_train_preprocessed.info())
behaviors_train_preprocessed.head()

<class 'pandas.core.frame.DataFrame'>
(5596624, 6)
<class 'pandas.core.frame.DataFrame'>
Index: 5596624 entries, 0 to 156964
Data columns (total 6 columns):
 #   Column             Dtype         
---  ------             -----         
 0   impression_id      int64         
 1   user_id            object        
 2   time               datetime64[ns]
 3   history            object        
 4   candidate_news_id  object        
 5   clicked            int64         
dtypes: datetime64[ns](1), int64(2), object(3)
memory usage: 298.9+ MB
None


impression_id user_id                time  \
0              1  U13740 2019-11-11 09:05:58   
0              1  U13740 2019-11-11 09:05:58   
1              2  U91836 2019-11-12 18:11:30   
1              2  U91836 2019-11-12 18:11:30   
1              2  U91836 2019-11-12 18:11:30   

                                             history candidate_news_id  \
0  [N55189, N42782, N34694, N45794, N18445, N6330...            N55689   
0  [N55189, N42782, N34694, N45794, N18445, N6330...            N35729   
1  [N31739, N6072, N63045, N23979, N43353, N8129,...            N20678   
1  [N31739, N6072, N63045, N23979, N43353, N8129,...            N39317   
1  [N31739, N6072, N63045, N23979, N43353, N8129,...            N20495   

   clicked  
0        1  
0        0  
1        0  
1        0  
1        0

In [ ]:
# TODO: find the date range of the train set
print("Earliest date in train set:", behaviors_train_preprocessed['time'].min())
print("Latest date in train set:", behaviors_train_preprocessed['time'].max())

Earliest date in train set: 2019-11-09 00:00:19
Latest date in train set: 2019-11-14 23:59:13


In [ ]:
# TODO: get number of unique users from train set and store them in a list to compare with val set
unique_users_train = behaviors_train_preprocessed['user_id'].unique().tolist()
print("Number of unique users in train set:", behaviors_train_preprocessed['user_id'].nunique())
print("Number of unique impression-user pairs in train set:", behaviors_train_preprocessed[['impression_id', 'user_id']].drop_duplicates().shape[0])

Number of unique users in train set: 50000
Number of unique impression-user pairs in train set: 156963


### Load news_train_with_item_vectors.parquet

In [ ]:
# REMINDER: This is use to load the data back to the original format from parquet files
try:
    news_train = pd.read_parquet('MINDsmall_train_processed/news_train_with_item_vectors.parquet')
except FileNotFoundError:
    from google.colab import files
    files.upload()
    news_train = pd.read_parquet('news_train_with_item_vectors.parquet')

Saving news_train_with_item_vectors.parquet to news_train_with_item_vectors.parquet


In [ ]:
print(type(news_train))
print(news_train.shape)
news_train.head(3)

<class 'pandas.core.frame.DataFrame'>
(51282, 11)


news_id   category      subcategory  \
0  N55528  lifestyle  lifestyleroyals   
1  N19639     health       weightloss   
2  N61837       news        newsworld   

                                               title  \
0  The Brands Queen Elizabeth, Prince Charles, an...   
1                      50 Worst Habits For Belly Fat   
2  The Cost of Trump's Aid Freeze in the Trenches...   

                                            abstract  \
0  Shop the notebooks, jackets, and more that the...   
1  These seemingly harmless habits are holding yo...   
2  Lt. Ivan Molchanets peeked over a parapet of s...   

                                             url  \
0  https://assets.msn.com/labs/mind/AAGH0ET.html   
1  https://assets.msn.com/labs/mind/AAB19MK.html   
2  https://assets.msn.com/labs/mind/AAJgNsz.html   

                                      title_entities  \
0  [{"Label": "Prince Philip, Duke of Edinburgh",...   
1  [{"Label": "Adipose tissue", "Type": "C", "Wik...   
2                                                 []   

                                   abstract_entities  \
0                                                 []   
1  [{"Label": "Adipose tissue", "Type": "C", "Wik...   
2  [{"Label": "Ukraine", "Type": "G", "WikidataId...   

                             item_vector_tfidf_dense  \
0  [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.7071067811865...   
1  [0.0, 0.0, 0.0, 0.0, 0.7071067811865475, 0.0, ...   
2  [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...   

                                     item_vector_w2v  \
0  [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.7071067811865...   
1  [0.0, 0.0, 0.0, 0.0, 0.7071067811865475, 0.0, ...   
2  [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...   

                                    item_vector_bert  
0  [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.7071067811865...  
1  [0.0, 0.0, 0.0, 0.0, 0.7071067811865475, 0.0, ...  
2  [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...

In [ ]:
# TODO: get the number of unique news_id and get list of unique news_id
unique_news_ids_train = news_train['news_id'].unique().tolist()
print("Number of unique news_id:", news_train['news_id'].nunique())

Number of unique news_id: 51282


In [ ]:
# TODO: get the number of unique category and subcategory and save a list for each
unique_categories = news_train['category'].unique().tolist()
unique_subcategories = news_train['subcategory'].unique().tolist()

print("Number of unique categories:", news_train['category'].nunique())
print("Number of unique subcategories:", news_train['subcategory'].nunique())

Number of unique categories: 17
Number of unique subcategories: 264


### Load behaviors_val.tsv

In [ ]:
try:
    behaviors_val = pd.read_csv('MINDsmall_val/behaviors_val.tsv', sep='\t', header=None,
                              names=['impression_id', 'user_id', 'time', 'history', 'impressions'])
except FileNotFoundError:
    from google.colab import files
    files.upload()
    behaviors_val = pd.read_csv('behaviors_val.tsv', sep='\t', header=None,
                              names=['impression_id', 'user_id', 'time', 'history', 'impressions'])

Saving behaviors_val.tsv to behaviors_val.tsv


In [ ]:
print(type(behaviors_val))
print(behaviors_val.shape)
print(behaviors_val.info())
behaviors_val.head()

<class 'pandas.core.frame.DataFrame'>
(73152, 5)
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 73152 entries, 0 to 73151
Data columns (total 5 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   impression_id  73152 non-null  int64 
 1   user_id        73152 non-null  object
 2   time           73152 non-null  object
 3   history        70938 non-null  object
 4   impressions    73152 non-null  object
dtypes: int64(1), object(4)
memory usage: 2.8+ MB
None


impression_id user_id                    time  \
0              1  U80234  11/15/2019 12:37:50 PM   
1              2  U60458   11/15/2019 7:11:50 AM   
2              3  U44190   11/15/2019 9:55:12 AM   
3              4  U87380   11/15/2019 3:12:46 PM   
4              5   U9444   11/15/2019 8:25:46 AM   

                                             history  \
0  N55189 N46039 N51741 N53234 N11276 N264 N40716...   
1  N58715 N32109 N51180 N33438 N54827 N28488 N611...   
2  N56253 N1150 N55189 N16233 N61704 N51706 N5303...   
3  N63554 N49153 N28678 N23232 N43369 N58518 N444...   
4                 N51692 N18285 N26015 N22679 N55556   

                                         impressions  
0  N28682-0 N48740-0 N31958-1 N34130-0 N6916-0 N5...  
1  N20036-0 N23513-1 N32536-0 N46976-0 N35216-0 N...  
2  N36779-0 N62365-0 N58098-0 N5472-0 N13408-0 N5...  
3  N6950-0 N60215-0 N6074-0 N11930-0 N6916-0 N248...  
4  N5940-1 N23513-0 N49285-0 N23355-0 N19990-0 N3...

In [ ]:
# Convert time to datetime
behaviors_val["time"] = pd.to_datetime(behaviors_val["time"])

In [ ]:
print(behaviors_val.isnull().sum())

impression_id       0
user_id             0
time                0
history          2214
impressions         0
dtype: int64


In [ ]:
# Fill NaNs for users with no history
print("Number of null values in history column:", behaviors_val["history"].isnull().sum())
behaviors_val["history"] = behaviors_val["history"].fillna("")
print("Number of null values in history column after fillna:", behaviors_val["history"].isnull().sum())

Number of null values in history column: 2214
Number of null values in history column after fillna: 0


In [ ]:
# Convert history string to list
behaviors_val["history"] = behaviors_val["history"].apply(lambda x: x.strip().split() if x else [])
print(type(behaviors_val.iloc[0]["history"]))
behaviors_val.head()

<class 'list'>


impression_id user_id                time  \
0              1  U80234 2019-11-15 12:37:50   
1              2  U60458 2019-11-15 07:11:50   
2              3  U44190 2019-11-15 09:55:12   
3              4  U87380 2019-11-15 15:12:46   
4              5   U9444 2019-11-15 08:25:46   

                                             history  \
0  [N55189, N46039, N51741, N53234, N11276, N264,...   
1  [N58715, N32109, N51180, N33438, N54827, N2848...   
2  [N56253, N1150, N55189, N16233, N61704, N51706...   
3  [N63554, N49153, N28678, N23232, N43369, N5851...   
4           [N51692, N18285, N26015, N22679, N55556]   

                                         impressions  
0  N28682-0 N48740-0 N31958-1 N34130-0 N6916-0 N5...  
1  N20036-0 N23513-1 N32536-0 N46976-0 N35216-0 N...  
2  N36779-0 N62365-0 N58098-0 N5472-0 N13408-0 N5...  
3  N6950-0 N60215-0 N6074-0 N11930-0 N6916-0 N248...  
4  N5940-1 N23513-0 N49285-0 N23355-0 N19990-0 N3...

In [ ]:
print(type(behaviors_val.iloc[0]["impressions"]))

<class 'str'>


In [ ]:
# Parse impressions into (news_id, label) pairs
def parse_impressions(imp_str):
    return [(item.split("-")[0], int(item.split("-")[1])) for item in imp_str.split()]

behaviors_val["impressions"] = behaviors_val["impressions"].apply(parse_impressions)
behaviors_val.head()

impression_id user_id                time  \
0              1  U80234 2019-11-15 12:37:50   
1              2  U60458 2019-11-15 07:11:50   
2              3  U44190 2019-11-15 09:55:12   
3              4  U87380 2019-11-15 15:12:46   
4              5   U9444 2019-11-15 08:25:46   

                                             history  \
0  [N55189, N46039, N51741, N53234, N11276, N264,...   
1  [N58715, N32109, N51180, N33438, N54827, N2848...   
2  [N56253, N1150, N55189, N16233, N61704, N51706...   
3  [N63554, N49153, N28678, N23232, N43369, N5851...   
4           [N51692, N18285, N26015, N22679, N55556]   

                                         impressions  
0  [(N28682, 0), (N48740, 0), (N31958, 1), (N3413...  
1  [(N20036, 0), (N23513, 1), (N32536, 0), (N4697...  
2  [(N36779, 0), (N62365, 0), (N58098, 0), (N5472...  
3  [(N6950, 0), (N60215, 0), (N6074, 0), (N11930,...  
4  [(N5940, 1), (N23513, 0), (N49285, 0), (N23355...

In [ ]:
print(type(behaviors_val.iloc[0]["impressions"]))       # should be list
print(type(behaviors_val.iloc[0]["impressions"][0]))    # should be tuple
print(type(behaviors_val.iloc[0]["impressions"][0][0])) # should be string
print(type(behaviors_val.iloc[0]["impressions"][0][1])) # should be ?

<class 'list'>
<class 'tuple'>
<class 'str'>
<class 'int'>


### Load news_val.tsv

In [ ]:
# Load news_val.tsv
try:
    news_val = pd.read_csv('MINDsmall_val/news_val.tsv', sep='\t', header=None,
                   names=['news_id', 'category', 'subcategory', 'title', 'abstract', 'url',
                          'title_entities', 'abstract_entities'], encoding='utf-8')
except FileNotFoundError:
    from google.colab import files
    files.upload()
    news_val = pd.read_csv('news_val.tsv', sep='\t', header=None,
                   names=['news_id', 'category', 'subcategory', 'title', 'abstract', 'url',
                          'title_entities', 'abstract_entities'], encoding='utf-8')

Saving news_val.tsv to news_val.tsv


In [ ]:
print(type(news_val))
print(news_val.shape)
print(news_val.info())
news_val.head()

<class 'pandas.core.frame.DataFrame'>
(42416, 8)
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 42416 entries, 0 to 42415
Data columns (total 8 columns):
 #   Column             Non-Null Count  Dtype 
---  ------             --------------  ----- 
 0   news_id            42416 non-null  object
 1   category           42416 non-null  object
 2   subcategory        42416 non-null  object
 3   title              42416 non-null  object
 4   abstract           40395 non-null  object
 5   url                42416 non-null  object
 6   title_entities     42414 non-null  object
 7   abstract_entities  42414 non-null  object
dtypes: object(8)
memory usage: 2.6+ MB
None


news_id   category      subcategory  \
0  N55528  lifestyle  lifestyleroyals   
1  N18955     health          medical   
2  N61837       news        newsworld   
3  N53526     health           voices   
4  N38324     health          medical   

                                               title  \
0  The Brands Queen Elizabeth, Prince Charles, an...   
1  Dispose of unwanted prescription drugs during ...   
2  The Cost of Trump's Aid Freeze in the Trenches...   
3  I Was An NBA Wife. Here's How It Affected My M...   
4  How to Get Rid of Skin Tags, According to a De...   

                                            abstract  \
0  Shop the notebooks, jackets, and more that the...   
1                                                NaN   
2  Lt. Ivan Molchanets peeked over a parapet of s...   
3  I felt like I was a fraud, and being an NBA wi...   
4  They seem harmless, but there's a very good re...   

                                             url  \
0  https://assets.msn.com/labs/mind/AAGH0ET.html   
1  https://assets.msn.com/labs/mind/AAISxPN.html   
2  https://assets.msn.com/labs/mind/AAJgNsz.html   
3  https://assets.msn.com/labs/mind/AACk2N6.html   
4  https://assets.msn.com/labs/mind/AAAKEkt.html   

                                      title_entities  \
0  [{"Label": "Prince Philip, Duke of Edinburgh",...   
1  [{"Label": "Drug Enforcement Administration", ...   
2                                                 []   
3                                                 []   
4  [{"Label": "Skin tag", "Type": "C", "WikidataI...   

                                   abstract_entities  
0                                                 []  
1                                                 []  
2  [{"Label": "Ukraine", "Type": "G", "WikidataId...  
3  [{"Label": "National Basketball Association", ...  
4  [{"Label": "Skin tag", "Type": "C", "WikidataI...

#### Data Cleaning (handle null values)

In [ ]:
news_val.isnull().sum()

news_id                 0
category                0
subcategory             0
title                   0
abstract             2021
url                     0
title_entities          2
abstract_entities       2
dtype: int64

In [ ]:
print(news_val.apply(lambda row: row.astype(str).isin(['']).any(), axis=1).sum())

0


In [ ]:
print(news_val.apply(lambda row: row.astype(str).isin(['[]']).any(), axis=1).sum())

17223


In [ ]:
print(f"Number of rows in 'title_entities' column containing []: {news_val['title_entities'].apply(lambda x: x == '[]').sum()}")

Number of rows in 'title_entities' column containing []: 11286


In [ ]:
print(f"Number of rows in 'abstract_entities' column containing []: {news_val['abstract_entities'].apply(lambda x: x == '[]').sum()}")

Number of rows in 'abstract_entities' column containing []: 11620


**There are quite a lot of rows with '[]' in the 'title_entities' and 'abstract entities' columns. Let's hold off on removing these for now. If we want to leverage the entity embeddings and relation embeddings, then we might need to remove these rows.**

If we drop news articles from news_val.tsv due to null values (e.g., missing abstract, title_entities, or abstract_entities), we must also remove any references to those articles from:

* the History column in behaviors_val.tsv

* the Impressions column in behaviors_val.tsv <br/>

Otherwise, our model may try to build user/item profiles based on missing data, which leads to inconsistencies or errors.

In [ ]:
# Drop incomplete articles from news (i.e., rows with NaN)
news_val = news_val.dropna(subset=["abstract", "title_entities", "abstract_entities"])
valid_news_ids = set(news_val["news_id"])
news_val.shape

(40393, 8)

In [ ]:
behaviors_val[behaviors_val["impressions"].map(len) == 0]

Empty DataFrame
Columns: [impression_id, user_id, time, history, impressions]
Index: []

In [ ]:
# Clean behaviors_val.tsv references
print(behaviors_val.shape)

# Remove invalid news from user history
behaviors_val["history"] = behaviors_val["history"].apply(
    lambda h: [nid for nid in h if nid in valid_news_ids]
)

# Remove invalid impressions
def clean_impressions(imps):
    return [(nid, label) for nid, label in imps if nid in valid_news_ids]

behaviors_val["impressions"] = behaviors_val["impressions"].apply(clean_impressions)

print(behaviors_val.shape)

(73152, 5)
(73152, 5)


In [ ]:
# Check number of rows with no impressions remaining after removing invalid impressions
behaviors_val[behaviors_val["impressions"].map(len) == 0]

Empty DataFrame
Columns: [impression_id, user_id, time, history, impressions]
Index: []

In [ ]:
# Remove rows with no impressions remaining
behaviors_val = behaviors_val[behaviors_val["impressions"].map(len) > 0]

In [ ]:
# Check number of rows with no impressions remaining after removing rows with no impressions remaining
behaviors_val[behaviors_val["impressions"].map(len) == 0]

Empty DataFrame
Columns: [impression_id, user_id, time, history, impressions]
Index: []

In [ ]:
behaviors_val.shape

(73152, 5)

In [ ]:
duplicate_count = news_val.duplicated(subset='news_id').sum()
print(f'Duplicated news_id entries: {duplicate_count}')

Duplicated news_id entries: 0


In [ ]:
print(news_val.shape)
news_val.head()

(40393, 8)


news_id   category      subcategory  \
0  N55528  lifestyle  lifestyleroyals   
2  N61837       news        newsworld   
3  N53526     health           voices   
4  N38324     health          medical   
5   N2073     sports     football_nfl   

                                               title  \
0  The Brands Queen Elizabeth, Prince Charles, an...   
2  The Cost of Trump's Aid Freeze in the Trenches...   
3  I Was An NBA Wife. Here's How It Affected My M...   
4  How to Get Rid of Skin Tags, According to a De...   
5  Should NFL be able to fine players for critici...   

                                            abstract  \
0  Shop the notebooks, jackets, and more that the...   
2  Lt. Ivan Molchanets peeked over a parapet of s...   
3  I felt like I was a fraud, and being an NBA wi...   
4  They seem harmless, but there's a very good re...   
5  Several fines came down against NFL players fo...   

                                             url  \
0  https://assets.msn.com/labs/mind/AAGH0ET.html   
2  https://assets.msn.com/labs/mind/AAJgNsz.html   
3  https://assets.msn.com/labs/mind/AACk2N6.html   
4  https://assets.msn.com/labs/mind/AAAKEkt.html   
5  https://assets.msn.com/labs/mind/AAJ4lap.html   

                                      title_entities  \
0  [{"Label": "Prince Philip, Duke of Edinburgh",...   
2                                                 []   
3                                                 []   
4  [{"Label": "Skin tag", "Type": "C", "WikidataI...   
5  [{"Label": "National Football League", "Type":...   

                                   abstract_entities  
0                                                 []  
2  [{"Label": "Ukraine", "Type": "G", "WikidataId...  
3  [{"Label": "National Basketball Association", ...  
4  [{"Label": "Skin tag", "Type": "C", "WikidataI...  
5  [{"Label": "National Football League", "Type":...

#### EDA

In [ ]:
# TODO: find the date range of the val set
print("Earliest date in validation set:", behaviors_val['time'].min())
print("Latest date in validation set:", behaviors_val['time'].max())

Earliest date in validation set: 2019-11-15 00:00:01
Latest date in validation set: 2019-11-15 23:58:03


In [ ]:
# TODO: get number of unique users from val set and compare with train set
unique_users_val = behaviors_val['user_id'].unique().tolist()
print("Number of unique users in validation set:", behaviors_val['user_id'].nunique())
print("Number of unique impression-user pairs in validation set:", behaviors_val[['impression_id', 'user_id']].drop_duplicates().shape[0])

Number of unique users in validation set: 50000
Number of unique impression-user pairs in validation set: 73152


In [ ]:
# TODO: find the number of different entries between unique_users_train and unique_users_val
set_train = set(unique_users_train)
set_val = set(unique_users_val)

# Symmetric difference: users in either list but not both
diff_users = set_train.symmetric_difference(set_val)

# Both
common_users = set_train & set_val

print("Length of set_train:", len(set_train))
print("Length of set_val:", len(set_val))
print("Number of different users between train and val:", len(diff_users))
print("Number of common users between train and val:", len(common_users))
print("Number of unique users in train or val (union):", len(set_train.union(set_val)))

Length of set_train: 50000
Length of set_val: 50000
Number of different users between train and val: 88114
Number of common users between train and val: 5943
Number of unique users in train or val (union): 94057


In [ ]:
# TODO: make sure that any news_id that is in news_train that appears in a user's history or impressions is also in news_val
unique_news_ids_val = news_val['news_id'].unique().tolist()
print("Number of unique news_id in val:", news_val['news_id'].nunique())

set_train = set(unique_news_ids_train)
set_val = set(unique_news_ids_val)

# Symmetric difference: news in either list but not both
diff_news = set_train.symmetric_difference(set_val)

# Both
common_news = set_train & set_val

print("Length of set_train:", len(set_train))
print("Length of set_val:", len(set_val))
print("Number of different news_id between train and val:", len(diff_news))
print("Number of common news_id between train and val:", len(common_news))
print("Number of unique news_id in train or val (union):", len(set_train.union(set_val)))

Number of unique news_id in val: 40393
Length of set_train: 51282
Length of set_val: 40393
Number of different news_id between train and val: 37303
Number of common news_id between train and val: 27186
Number of unique news_id in train or val (union): 64489


In [ ]:
# TODO: make sure that any news_id that appears in a user's history or impressions in news_val is also in news_val
# valid_news_ids: the set of all valid news_ids from news_val

# Extract all news_ids from 'history' column
history_news_ids = set()
for history in behaviors_val['history']:
    history_news_ids.update(history)  # assumes each history is already a list

# Extract all news_ids from 'impressions' column
impression_news_ids = set()
for impressions in behaviors_val['impressions']:
    for item in impressions:
        impression_news_ids.add(item[0])  # extract news_id (first element of the tuple)

# Combine both sets
all_behavior_news_ids = set(history_news_ids.union(impression_news_ids))

# Find missing news_ids
missing_news_ids = all_behavior_news_ids - valid_news_ids

# Print result
if missing_news_ids:
    print(f"News IDs in behaviors_val but NOT in news_val: {missing_news_ids}")
    print(f"Total missing news_ids: {len(missing_news_ids)}")
else:
    print("All news_ids in behaviors_val are present in news_val.")

All news_ids in behaviors_val are present in news_val.


#### Text Preprocessing

In [ ]:
import re
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
import string

In [ ]:
nltk.download('stopwords')
nltk.download('punkt_tab')
nltk.download('wordnet')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt_tab.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...


True

In [ ]:
# Concatenate the content of the 'title' and 'abstract' columns into a column called 'text'
news_val['text'] = news_val['title'] + " " + news_val['abstract']

lemmatizer = WordNetLemmatizer()
stop_words = set(stopwords.words('english'))

In [ ]:
def clean_and_tokenize_text(text):
  if not isinstance(text, str):
      text = str(text)

  # Removes all characters from the text that are not lowercase letters or whitespace (punctuation, numbers, special symbols)
  text = re.sub(r'[^a-z\s]', '', text)

  text = text.lower() # Make text lowercase

  tokens = word_tokenize(text) # Split the cleaned text into a list of individual words (tokens)

  cleaned_tokens = [lemmatizer.lemmatize(word) for word in tokens if word not in stop_words]

  return cleaned_tokens

In [ ]:
# Apply to 'text' column
news_val['text_tokens'] = news_val['text'].apply(clean_and_tokenize_text)

In [ ]:
# Apply to title and abstract
news_val["title_tokens"] = news_val["title"].apply(clean_and_tokenize_text)
news_val["abstract_tokens"] = news_val["abstract"].apply(clean_and_tokenize_text)

In [ ]:
# For TF-IDF: Join tokens back into text
news_val["text_tfidf"] = news_val["text_tokens"].apply(lambda tokens: " ".join(tokens))

In [ ]:
# For Word2Vec: Keep tokenized version
news_val["text_w2v"] = news_val["text_tokens"]

In [ ]:
# Combine title and abstract as a single string for BERT input
def combine_for_bert(row):
    title = str(row["title"]) #if pd.notnull(row["title"]) else ""
    abstract = str(row["abstract"]) #if pd.notnull(row["abstract"]) else ""
    return title.strip() + " " + abstract.strip()

In [ ]:
# Combine title and abstract as a single string for BERT input
news_val["text_bert"] = news_val.apply(combine_for_bert, axis=1)

In [ ]:
print(news_val.shape)
news_val.head()

(40393, 15)


news_id   category      subcategory  \
0  N55528  lifestyle  lifestyleroyals   
2  N61837       news        newsworld   
3  N53526     health           voices   
4  N38324     health          medical   
5   N2073     sports     football_nfl   

                                               title  \
0  The Brands Queen Elizabeth, Prince Charles, an...   
2  The Cost of Trump's Aid Freeze in the Trenches...   
3  I Was An NBA Wife. Here's How It Affected My M...   
4  How to Get Rid of Skin Tags, According to a De...   
5  Should NFL be able to fine players for critici...   

                                            abstract  \
0  Shop the notebooks, jackets, and more that the...   
2  Lt. Ivan Molchanets peeked over a parapet of s...   
3  I felt like I was a fraud, and being an NBA wi...   
4  They seem harmless, but there's a very good re...   
5  Several fines came down against NFL players fo...   

                                             url  \
0  https://assets.msn.com/labs/mind/AAGH0ET.html   
2  https://assets.msn.com/labs/mind/AAJgNsz.html   
3  https://assets.msn.com/labs/mind/AACk2N6.html   
4  https://assets.msn.com/labs/mind/AAAKEkt.html   
5  https://assets.msn.com/labs/mind/AAJ4lap.html   

                                      title_entities  \
0  [{"Label": "Prince Philip, Duke of Edinburgh",...   
2                                                 []   
3                                                 []   
4  [{"Label": "Skin tag", "Type": "C", "WikidataI...   
5  [{"Label": "National Football League", "Type":...   

                                   abstract_entities  \
0                                                 []   
2  [{"Label": "Ukraine", "Type": "G", "WikidataId...   
3  [{"Label": "National Basketball Association", ...   
4  [{"Label": "Skin tag", "Type": "C", "WikidataI...   
5  [{"Label": "National Football League", "Type":...   

                                                text  \
0  The Brands Queen Elizabeth, Prince Charles, an...   
2  The Cost of Trump's Aid Freeze in the Trenches...   
3  I Was An NBA Wife. Here's How It Affected My M...   
4  How to Get Rid of Skin Tags, According to a De...   
5  Should NFL be able to fine players for critici...   

                                         text_tokens  \
0  [rand, ueen, lizabeth, rince, harles, rince, h...   
2  [ost, rump, id, reeze, renches, kraines, ar, v...   
3  [n, ife, eres, ow, ffected, ental, ealth, felt...   
4  [ow, et, id, kin, ag, ccording, ermatologist, ...   
5  [hould, able, fine, player, criticizing, offic...   

                                        title_tokens  \
0  [rand, ueen, lizabeth, rince, harles, rince, h...   
2       [ost, rump, id, reeze, renches, kraines, ar]   
3          [n, ife, eres, ow, ffected, ental, ealth]   
4      [ow, et, id, kin, ag, ccording, ermatologist]   
5  [hould, able, fine, player, criticizing, offic...   

                                     abstract_tokens  \
0  [hop, notebook, jacket, royal, cant, live, wit...   
2  [van, olchanets, peeked, parapet, sand, bag, f...   
3  [felt, like, fraud, wife, didnt, help, n, fact...   
4  [hey, seem, harmless, there, good, reason, sho...   
5  [everal, fine, came, player, criticizing, offi...   

                                          text_tfidf  \
0  rand ueen lizabeth rince harles rince hilip we...   
2  ost rump id reeze renches kraines ar van olcha...   
3  n ife eres ow ffected ental ealth felt like fr...   
4  ow et id kin ag ccording ermatologist hey seem...   
5  hould able fine player criticizing officiating...   

                                            text_w2v  \
0  [rand, ueen, lizabeth, rince, harles, rince, h...   
2  [ost, rump, id, reeze, renches, kraines, ar, v...   
3  [n, ife, eres, ow, ffected, ental, ealth, felt...   
4  [ow, et, id, kin, ag, ccording, ermatologist, ...   
5  [hould, able, fine, player, criticizing, offic...   

                                           text_bert  
0  The Brands Que

### Checkpoint: Save news_val

In [ ]:
print(type(news_val))
print(news_val.shape)
news_val.head(2)

<class 'pandas.core.frame.DataFrame'>
(40393, 15)


news_id   category      subcategory  \
0  N55528  lifestyle  lifestyleroyals   
2  N61837       news        newsworld   

                                               title  \
0  The Brands Queen Elizabeth, Prince Charles, an...   
2  The Cost of Trump's Aid Freeze in the Trenches...   

                                            abstract  \
0  Shop the notebooks, jackets, and more that the...   
2  Lt. Ivan Molchanets peeked over a parapet of s...   

                                             url  \
0  https://assets.msn.com/labs/mind/AAGH0ET.html   
2  https://assets.msn.com/labs/mind/AAJgNsz.html   

                                      title_entities  \
0  [{"Label": "Prince Philip, Duke of Edinburgh",...   
2                                                 []   

                                   abstract_entities  \
0                                                 []   
2  [{"Label": "Ukraine", "Type": "G", "WikidataId...   

                                                text  \
0  The Brands Queen Elizabeth, Prince Charles, an...   
2  The Cost of Trump's Aid Freeze in the Trenches...   

                                         text_tokens  \
0  [rand, ueen, lizabeth, rince, harles, rince, h...   
2  [ost, rump, id, reeze, renches, kraines, ar, v...   

                                        title_tokens  \
0  [rand, ueen, lizabeth, rince, harles, rince, h...   
2       [ost, rump, id, reeze, renches, kraines, ar]   

                                     abstract_tokens  \
0  [hop, notebook, jacket, royal, cant, live, wit...   
2  [van, olchanets, peeked, parapet, sand, bag, f...   

                                          text_tfidf  \
0  rand ueen lizabeth rince harles rince hilip we...   
2  ost rump id reeze renches kraines ar van olcha...   

                                            text_w2v  \
0  [rand, ueen, lizabeth, rince, harles, rince, h...   
2  [ost, rump, id, reeze, renches, kraines, ar, v...   

                                           text_bert  
0  The Brands Queen Elizabeth, Prince Charles, an...  
2  The Cost of Trump's Aid Freeze in the Trenches...

In [ ]:
# Define the filename for the downloaded parquet
filename = 'news_val_tokenized.parquet'

# Save the DataFrame to a temporary parquet file in Colab's environment
news_val.to_parquet(filename, index=False)

# Trigger the download
try:
    files.download(filename)
    print(f"DataFrame downloaded as: {filename}")
except:
    pass

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

DataFrame downloaded as: news_val_tokenized.parquet


### Checkpoint: Explode and save behaviors_val

In [ ]:
behaviors_val_exploded = behaviors_val.explode('impressions').dropna()
behaviors_val_exploded['candidate_news_id'] = behaviors_val_exploded['impressions'].apply(lambda x: x[0])
behaviors_val_exploded['clicked'] = behaviors_val_exploded['impressions'].apply(lambda x: x[1])

In [ ]:
behaviors_val_exploded = behaviors_val_exploded.drop(columns='impressions')

In [ ]:
print("behaviors_val shape:", behaviors_val.shape)
behaviors_val.head()

behaviors_val shape: (73152, 5)


impression_id user_id                time  \
0              1  U80234 2019-11-15 12:37:50   
1              2  U60458 2019-11-15 07:11:50   
2              3  U44190 2019-11-15 09:55:12   
3              4  U87380 2019-11-15 15:12:46   
4              5   U9444 2019-11-15 08:25:46   

                                             history  \
0  [N55189, N46039, N51741, N53234, N11276, N264,...   
1  [N58715, N32109, N51180, N33438, N54827, N2848...   
2  [N56253, N1150, N55189, N16233, N61704, N51706...   
3  [N63554, N49153, N28678, N43369, N58518, N4440...   
4           [N51692, N18285, N26015, N22679, N55556]   

                                         impressions  
0  [(N28682, 0), (N48740, 0), (N31958, 1), (N3413...  
1  [(N20036, 0), (N32536, 0), (N46976, 0), (N3521...  
2  [(N36779, 0), (N62365, 0), (N58098, 0), (N5472...  
3  [(N6950, 0), (N60215, 0), (N6074, 0), (N11930,...  
4  [(N5940, 1), (N49285, 0), (N23355, 0), (N19990...

In [ ]:
print("behaviors_val_exploded shape:", behaviors_val_exploded.shape)
behaviors_val_exploded.head()

behaviors_val_exploded shape: (2670611, 6)


impression_id user_id                time  \
0              1  U80234 2019-11-15 12:37:50   
0              1  U80234 2019-11-15 12:37:50   
0              1  U80234 2019-11-15 12:37:50   
0              1  U80234 2019-11-15 12:37:50   
0              1  U80234 2019-11-15 12:37:50   

                                             history candidate_news_id  \
0  [N55189, N46039, N51741, N53234, N11276, N264,...            N28682   
0  [N55189, N46039, N51741, N53234, N11276, N264,...            N48740   
0  [N55189, N46039, N51741, N53234, N11276, N264,...            N31958   
0  [N55189, N46039, N51741, N53234, N11276, N264,...            N34130   
0  [N55189, N46039, N51741, N53234, N11276, N264,...             N6916   

   clicked  
0        0  
0        0  
0        1  
0        0  
0        0

In [ ]:
# Define the filename for the downloaded parquet
filename = 'behaviors_val_preprocessed.parquet'

# Save the DataFrame to a temporary parquet file in Colab's environment
behaviors_val_exploded.to_parquet(filename)

# Trigger the download
try:
    files.download(filename)
    print(f"DataFrame downloaded as: {filename}")
except:
    pass

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

DataFrame downloaded as: behaviors_val_preprocessed.parquet


### Text Processing

In [1]:
import numpy as np
import pandas as pd

#### Load preprocessed news_val_tokenized.parquet

In [3]:
# REMINDER: This is use to load the data back to the original format from parquet files
try:
    news_val_tokenized = pd.read_parquet('MINDsmall_val_processed/news_val_tokenized.parquet')
except FileNotFoundError:
    from google.colab import files
    files.upload()
    news_val_tokenized = pd.read_parquet('news_val_tokenized.parquet')

Saving news_val_tokenized.parquet to news_val_tokenized.parquet


In [4]:
print(type(news_val_tokenized))
print(news_val_tokenized.shape)
news_val_tokenized.head()

<class 'pandas.core.frame.DataFrame'>
(40393, 15)


news_id   category      subcategory  \
0  N55528  lifestyle  lifestyleroyals   
1  N61837       news        newsworld   
2  N53526     health           voices   
3  N38324     health          medical   
4   N2073     sports     football_nfl   

                                               title  \
0  The Brands Queen Elizabeth, Prince Charles, an...   
1  The Cost of Trump's Aid Freeze in the Trenches...   
2  I Was An NBA Wife. Here's How It Affected My M...   
3  How to Get Rid of Skin Tags, According to a De...   
4  Should NFL be able to fine players for critici...   

                                            abstract  \
0  Shop the notebooks, jackets, and more that the...   
1  Lt. Ivan Molchanets peeked over a parapet of s...   
2  I felt like I was a fraud, and being an NBA wi...   
3  They seem harmless, but there's a very good re...   
4  Several fines came down against NFL players fo...   

                                             url  \
0  https://assets.msn.com/labs/mind/AAGH0ET.html   
1  https://assets.msn.com/labs/mind/AAJgNsz.html   
2  https://assets.msn.com/labs/mind/AACk2N6.html   
3  https://assets.msn.com/labs/mind/AAAKEkt.html   
4  https://assets.msn.com/labs/mind/AAJ4lap.html   

                                      title_entities  \
0  [{"Label": "Prince Philip, Duke of Edinburgh",...   
1                                                 []   
2                                                 []   
3  [{"Label": "Skin tag", "Type": "C", "WikidataI...   
4  [{"Label": "National Football League", "Type":...   

                                   abstract_entities  \
0                                                 []   
1  [{"Label": "Ukraine", "Type": "G", "WikidataId...   
2  [{"Label": "National Basketball Association", ...   
3  [{"Label": "Skin tag", "Type": "C", "WikidataI...   
4  [{"Label": "National Football League", "Type":...   

                                                text  \
0  The Brands Queen Elizabeth, Prince Charles, an...   
1  The Cost of Trump's Aid Freeze in the Trenches...   
2  I Was An NBA Wife. Here's How It Affected My M...   
3  How to Get Rid of Skin Tags, According to a De...   
4  Should NFL be able to fine players for critici...   

                                         text_tokens  \
0  [rand, ueen, lizabeth, rince, harles, rince, h...   
1  [ost, rump, id, reeze, renches, kraines, ar, v...   
2  [n, ife, eres, ow, ffected, ental, ealth, felt...   
3  [ow, et, id, kin, ag, ccording, ermatologist, ...   
4  [hould, able, fine, player, criticizing, offic...   

                                        title_tokens  \
0  [rand, ueen, lizabeth, rince, harles, rince, h...   
1       [ost, rump, id, reeze, renches, kraines, ar]   
2          [n, ife, eres, ow, ffected, ental, ealth]   
3      [ow, et, id, kin, ag, ccording, ermatologist]   
4  [hould, able, fine, player, criticizing, offic...   

                                     abstract_tokens  \
0  [hop, notebook, jacket, royal, cant, live, wit...   
1  [van, olchanets, peeked, parapet, sand, bag, f...   
2  [felt, like, fraud, wife, didnt, help, n, fact...   
3  [hey, seem, harmless, there, good, reason, sho...   
4  [everal, fine, came, player, criticizing, offi...   

                                          text_tfidf  \
0  rand ueen lizabeth rince harles rince hilip we...   
1  ost rump id reeze renches kraines ar van olcha...   
2  n ife eres ow ffected ental ealth felt like fr...   
3  ow et id kin ag ccording ermatologist hey seem...   
4  hould able fine player criticizing officiating...   

                                            text_w2v  \
0  [rand, ueen, lizabeth, rince, harles, rince, h...   
1  [ost, rump, id, reeze, renches, kraines, ar, v...   
2  [n, ife, eres, ow, ffected, ental, ealth, felt...   
3  [ow, et, id, kin, ag, ccording, ermatologist, ...   
4  [hould, able, fine, player, criticizing, offic...   

                                           text_bert  
0  The Brands Que

#### One-hot-encode Category and Subcategory to get binary vectors

In [5]:
print(news_val_tokenized['category'].nunique())
news_val_tokenized['category'].value_counts()

17


category
news             12596
sports           10777
finance           2533
foodanddrink      2213
lifestyle         2027
travel            1755
health            1676
video             1598
weather           1364
autos             1364
tv                 807
music              605
movies             537
entertainment      528
kids                10
middleeast           2
games                1
Name: count, dtype: int64

In [6]:
print(news_val_tokenized['subcategory'].nunique())
news_val_tokenized['subcategory'].value_counts()

250


subcategory
newsus             5125
football_nfl       4326
newspolitics       2335
newscrime          1708
newsworld          1476
                   ... 
healthyliving         1
lifestylestyle        1
health-cancer         1
autosmidsize          1
autoslosangeles       1
Name: count, Length: 250, dtype: int64

In [7]:
# TODO:
category_onehot = pd.get_dummies(news_val_tokenized['category'], prefix='cat')
subcategory_onehot = pd.get_dummies(news_val_tokenized['subcategory'], prefix='subcat')

news_val_tokenized = pd.concat([news_val_tokenized, category_onehot, subcategory_onehot], axis=1)

print("News DataFrame with One-Hot Encoded Category/Subcategory:")
print(news_val_tokenized.shape)

News DataFrame with One-Hot Encoded Category/Subcategory:
(40393, 282)


In [8]:
news_val_tokenized.head(1)

news_id   category      subcategory  \
0  N55528  lifestyle  lifestyleroyals   

                                               title  \
0  The Brands Queen Elizabeth, Prince Charles, an...   

                                            abstract  \
0  Shop the notebooks, jackets, and more that the...   

                                             url  \
0  https://assets.msn.com/labs/mind/AAGH0ET.html   

                                      title_entities abstract_entities  \
0  [{"Label": "Prince Philip, Duke of Edinburgh",...                []   

                                                text  \
0  The Brands Queen Elizabeth, Prince Charles, an...   

                                         text_tokens  ... subcat_viral  \
0  [rand, ueen, lizabeth, rince, harles, rince, h...  ...        False   

  subcat_voices subcat_weatherfullscreenmaps subcat_weathertopstories  \
0         False                        False                    False   

  subcat_weight-loss  subcat_weightloss  subcat_wellness  subcat_wines  \
0              False              False            False         False   

   subcat_wonder  subcat_yearinoffbeatgoodnews  
0          False                         False  

[1 rows x 282 columns]

In [9]:
# Get all column names that start with 'cat_' or 'subcat_'
onehot_cols = [col for col in news_val_tokenized.columns if col.startswith('cat_') or col.startswith('subcat_')]

# Create a column that stores binary vector
news_val_tokenized['vector_cat_subcat'] = news_val_tokenized[onehot_cols].values.tolist()

In [10]:
news_val_tokenized.head(1)

news_id   category      subcategory  \
0  N55528  lifestyle  lifestyleroyals   

                                               title  \
0  The Brands Queen Elizabeth, Prince Charles, an...   

                                            abstract  \
0  Shop the notebooks, jackets, and more that the...   

                                             url  \
0  https://assets.msn.com/labs/mind/AAGH0ET.html   

                                      title_entities abstract_entities  \
0  [{"Label": "Prince Philip, Duke of Edinburgh",...                []   

                                                text  \
0  The Brands Queen Elizabeth, Prince Charles, an...   

                                         text_tokens  ... subcat_voices  \
0  [rand, ueen, lizabeth, rince, harles, rince, h...  ...         False   

  subcat_weatherfullscreenmaps subcat_weathertopstories subcat_weight-loss  \
0                        False                    False              False   

  subcat_weightloss  subcat_wellness  subcat_wines  subcat_wonder  \
0             False            False         False          False   

   subcat_yearinoffbeatgoodnews  \
0                         False   

                                   vector_cat_subcat  
0  [False, False, False, False, False, False, Fal...  

[1 rows x 283 columns]

In [11]:
# Convert booleans to 0/1s
news_val_tokenized['vector_cat_subcat'] = news_val_tokenized['vector_cat_subcat'].apply(
    lambda x: [int(val) for val in x]
)

In [12]:
# Drop the ohe columns
columns_to_drop = [col for col in news_val_tokenized.columns if col.startswith('cat_') or col.startswith('subcat_')]
news_val_tokenized.drop(columns=columns_to_drop, inplace=True)

In [13]:
print(news_val_tokenized.shape)
news_val_tokenized.head(3)

(40393, 16)


news_id   category      subcategory  \
0  N55528  lifestyle  lifestyleroyals   
1  N61837       news        newsworld   
2  N53526     health           voices   

                                               title  \
0  The Brands Queen Elizabeth, Prince Charles, an...   
1  The Cost of Trump's Aid Freeze in the Trenches...   
2  I Was An NBA Wife. Here's How It Affected My M...   

                                            abstract  \
0  Shop the notebooks, jackets, and more that the...   
1  Lt. Ivan Molchanets peeked over a parapet of s...   
2  I felt like I was a fraud, and being an NBA wi...   

                                             url  \
0  https://assets.msn.com/labs/mind/AAGH0ET.html   
1  https://assets.msn.com/labs/mind/AAJgNsz.html   
2  https://assets.msn.com/labs/mind/AACk2N6.html   

                                      title_entities  \
0  [{"Label": "Prince Philip, Duke of Edinburgh",...   
1                                                 []   
2                                                 []   

                                   abstract_entities  \
0                                                 []   
1  [{"Label": "Ukraine", "Type": "G", "WikidataId...   
2  [{"Label": "National Basketball Association", ...   

                                                text  \
0  The Brands Queen Elizabeth, Prince Charles, an...   
1  The Cost of Trump's Aid Freeze in the Trenches...   
2  I Was An NBA Wife. Here's How It Affected My M...   

                                         text_tokens  \
0  [rand, ueen, lizabeth, rince, harles, rince, h...   
1  [ost, rump, id, reeze, renches, kraines, ar, v...   
2  [n, ife, eres, ow, ffected, ental, ealth, felt...   

                                        title_tokens  \
0  [rand, ueen, lizabeth, rince, harles, rince, h...   
1       [ost, rump, id, reeze, renches, kraines, ar]   
2          [n, ife, eres, ow, ffected, ental, ealth]   

                                     abstract_tokens  \
0  [hop, notebook, jacket, royal, cant, live, wit...   
1  [van, olchanets, peeked, parapet, sand, bag, f...   
2  [felt, like, fraud, wife, didnt, help, n, fact...   

                                          text_tfidf  \
0  rand ueen lizabeth rince harles rince hilip we...   
1  ost rump id reeze renches kraines ar van olcha...   
2  n ife eres ow ffected ental ealth felt like fr...   

                                            text_w2v  \
0  [rand, ueen, lizabeth, rince, harles, rince, h...   
1  [ost, rump, id, reeze, renches, kraines, ar, v...   
2  [n, ife, eres, ow, ffected, ental, ealth, felt...   

                                           text_bert  \
0  The Brands Queen Elizabeth, Prince Charles, an...   
1  The Cost of Trump's Aid Freeze in the Trenches...   
2  I Was An NBA Wife. Here's How It Affected My M...   

                                   vector_cat_subcat  
0  [0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, ...  
1  [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, ...  
2  [0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...

#### Get vectors/embeddings for Title and Abstract using TF-IDF

In [14]:
# TODO: load saved TFIDF model
import joblib
try:
    tfidf_vectorizer = joblib.load('model/tfidf_vectorizer.pkl')
except FileNotFoundError:
    from google.colab import files
    files.upload()
    tfidf_vectorizer = joblib.load('tfidf_vectorizer.pkl')

Saving tfidf_vectorizer.pkl to tfidf_vectorizer.pkl


In [15]:
print(tfidf_vectorizer.get_feature_names_out()[:5])

['ab' 'abandoned' 'abbard' 'abe' 'ability']


In [16]:
print(f"Val loaded vocabulary size: {len(tfidf_vectorizer.vocabulary_)}")
print(f"Val loaded vocabulary example (first 5): {list(tfidf_vectorizer.vocabulary_.items())[:5]}")
print(f"Val loaded IDF example (first 5): {tfidf_vectorizer.idf_[:5]}")

Val loaded vocabulary size: 5000
Val loaded vocabulary example (first 5): [('rand', 3620), ('ueen', 4632), ('lizabeth', 2598), ('rince', 3852), ('harles', 1959)]
Val loaded IDF example (first 5): [8.06092496 7.43839534 8.23419668 7.89387087 7.26014711]


In [17]:
# TODO: Get vectors/embeddings for Title and Abstract using TF-IDF and store vectors in a new column
from sklearn.feature_extraction.text import TfidfVectorizer

corpus = news_val_tokenized['text_tfidf'].astype(str).tolist()

#tfidf_vectorizer = TfidfVectorizer(max_features=5000)

tfidf_matrix = tfidf_vectorizer.transform(corpus)

news_val_tokenized['vector_tfidf'] = list(tfidf_matrix)

print(f"TF-IDF matrix shape: {tfidf_matrix.shape}")
print(news_val_tokenized[['news_id', 'vector_tfidf']].head(3))

TF-IDF matrix shape: (40393, 5000)
  news_id                                       vector_tfidf
0  N55528    (0, 726)\t0.2356566551182428\n  (0, 1959)\t0...
1  N61837    (0, 186)\t0.21543184635144275\n  (0, 321)\t0...
2  N53526    (0, 1132)\t0.3346893707812978\n  (0, 1148)\t...


In [18]:
tfidf_matrix

<40393x5000 sparse matrix of type '<class 'numpy.float64'>'
	with 751445 stored elements in Compressed Sparse Row format>

In [19]:
news_val_tokenized.head(2)

news_id   category      subcategory  \
0  N55528  lifestyle  lifestyleroyals   
1  N61837       news        newsworld   

                                               title  \
0  The Brands Queen Elizabeth, Prince Charles, an...   
1  The Cost of Trump's Aid Freeze in the Trenches...   

                                            abstract  \
0  Shop the notebooks, jackets, and more that the...   
1  Lt. Ivan Molchanets peeked over a parapet of s...   

                                             url  \
0  https://assets.msn.com/labs/mind/AAGH0ET.html   
1  https://assets.msn.com/labs/mind/AAJgNsz.html   

                                      title_entities  \
0  [{"Label": "Prince Philip, Duke of Edinburgh",...   
1                                                 []   

                                   abstract_entities  \
0                                                 []   
1  [{"Label": "Ukraine", "Type": "G", "WikidataId...   

                                                text  \
0  The Brands Queen Elizabeth, Prince Charles, an...   
1  The Cost of Trump's Aid Freeze in the Trenches...   

                                         text_tokens  \
0  [rand, ueen, lizabeth, rince, harles, rince, h...   
1  [ost, rump, id, reeze, renches, kraines, ar, v...   

                                        title_tokens  \
0  [rand, ueen, lizabeth, rince, harles, rince, h...   
1       [ost, rump, id, reeze, renches, kraines, ar]   

                                     abstract_tokens  \
0  [hop, notebook, jacket, royal, cant, live, wit...   
1  [van, olchanets, peeked, parapet, sand, bag, f...   

                                          text_tfidf  \
0  rand ueen lizabeth rince harles rince hilip we...   
1  ost rump id reeze renches kraines ar van olcha...   

                                            text_w2v  \
0  [rand, ueen, lizabeth, rince, harles, rince, h...   
1  [ost, rump, id, reeze, renches, kraines, ar, v...   

                                           text_bert  \
0  The Brands Queen Elizabeth, Prince Charles, an...   
1  The Cost of Trump's Aid Freeze in the Trenches...   

                                   vector_cat_subcat  \
0  [0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, ...   
1  [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, ...   

                                        vector_tfidf  
0    (0, 726)\t0.2356566551182428\n  (0, 1959)\t0...  
1    (0, 186)\t0.21543184635144275\n  (0, 321)\t0...

In [20]:
print(news_val_tokenized.shape)
news_val_tokenized['vector_tfidf'][0].toarray()

(40393, 17)


array([[0., 0., 0., ..., 0., 0., 0.]])

#### Get vectors/embeddings Title and Abstract using Word2Vec (Gensim)

In [1]:
pip install gensim

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.0/61.0 kB 5.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.6/60.6 kB 6.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 26.7/26.7 MB 16.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 18.3/18.3 MB 45.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 38.6/38.6 MB 14.3 MB/s eta 0:00:00
  Attempting uninstall: numpy
    Found existing installation: numpy 2.0.2
    Uninstalling numpy-2.0.2:
      Successfully uninstalled numpy-2.0.2
  Attempting uninstall: scipy
    Found existing installation: scipy 1.15.3
    Uninstalling scipy-1.15.3:
      Successfully uninstalled scipy-1.15.3
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
opencv-python-headless 4.12.0.88 requires numpy<2.3.0,>=2; python_version >= "3.9", but you have numpy 1.26.4 which is incompatib

In [21]:
# TODO: load saved w2v model
from gensim.models import Word2Vec
try:
    w2v_model = Word2Vec.load('models/word2vec.model')
except FileNotFoundError:
    from google.colab import files
    files.upload()
    w2v_model = Word2Vec.load('word2vec.model')

Saving word2vec.model to word2vec.model


In [22]:
type(news_val_tokenized['text_w2v'].tolist())

list

In [23]:
# debug Return True if any token is not a plain str
def is_bad(doc):
    if not isinstance(doc, list):
        return True
    for tok in doc:
        if not isinstance(tok, str):
            return True
    return False

bad_rows = news_val_tokenized["text_w2v"].apply(is_bad)
print(f"Bad rows: {bad_rows.sum()} / {len(news_val_tokenized)}")

Bad rows: 40393 / 40393


In [24]:
idx = bad_rows.idxmax()      # first bad row
print(news_val_tokenized.loc[idx, "text_w2v"])

['rand' 'ueen' 'lizabeth' 'rince' 'harles' 'rince' 'hilip' 'wear' 'hop'
 'notebook' 'jacket' 'royal' 'cant' 'live' 'without']


In [25]:
def sanitize(doc):
    # make sure we have a list
    if isinstance(doc, np.ndarray):
        doc = doc.tolist()
    elif not isinstance(doc, list):
        doc = list(doc) if doc is not None else []

    # keep only string tokens
    return [str(tok) for tok in doc if isinstance(tok, str)]

news_val_tokenized["text_w2v"] = news_val_tokenized["text_w2v"].apply(sanitize)

In [26]:
bad_rows = news_val_tokenized["text_w2v"].apply(is_bad)
print(f"Bad rows: {bad_rows.sum()} / {len(news_val_tokenized)}")

Bad rows: 0 / 40393


In [27]:
print(news_val_tokenized.loc[0, "text_w2v"])

['rand', 'ueen', 'lizabeth', 'rince', 'harles', 'rince', 'hilip', 'wear', 'hop', 'notebook', 'jacket', 'royal', 'cant', 'live', 'without']


In [28]:
# TODO: Get vectors/embeddings Title and Abstract using Word2Vec (Gensim) and store vectors in a new column
import gensim
from gensim.models import Word2Vec

sentences = news_val_tokenized['text_w2v'].tolist()

#w2v_model = Word2Vec(sentences, vector_size=100, window=5, min_count=2, workers=4, sg=1)

def get_w2v_vector(tokens, model):
    vectors = []
    for token in tokens:
        if token in model.wv:
            vectors.append(model.wv[token])
    if vectors:
        return np.mean(vectors, axis=0)
    else:
        return np.zeros(model.vector_size)

news_val_tokenized['vector_w2v'] = news_val_tokenized['text_tokens'].apply(lambda tokens: get_w2v_vector(tokens, w2v_model))

print(news_val_tokenized[['news_id', 'vector_w2v']].head(3))

  news_id                                         vector_w2v
0  N55528  [0.047399126, -0.09589149, -0.13837357, 0.0836...
1  N61837  [0.030284544, -0.030569091, -0.072350785, 0.10...
2  N53526  [0.22475731, -0.1408289, -0.0958124, -0.100263...


In [29]:
print(news_val_tokenized.shape)
news_val_tokenized.head(2)

(40393, 18)


news_id   category      subcategory  \
0  N55528  lifestyle  lifestyleroyals   
1  N61837       news        newsworld   

                                               title  \
0  The Brands Queen Elizabeth, Prince Charles, an...   
1  The Cost of Trump's Aid Freeze in the Trenches...   

                                            abstract  \
0  Shop the notebooks, jackets, and more that the...   
1  Lt. Ivan Molchanets peeked over a parapet of s...   

                                             url  \
0  https://assets.msn.com/labs/mind/AAGH0ET.html   
1  https://assets.msn.com/labs/mind/AAJgNsz.html   

                                      title_entities  \
0  [{"Label": "Prince Philip, Duke of Edinburgh",...   
1                                                 []   

                                   abstract_entities  \
0                                                 []   
1  [{"Label": "Ukraine", "Type": "G", "WikidataId...   

                                                text  \
0  The Brands Queen Elizabeth, Prince Charles, an...   
1  The Cost of Trump's Aid Freeze in the Trenches...   

                                         text_tokens  \
0  [rand, ueen, lizabeth, rince, harles, rince, h...   
1  [ost, rump, id, reeze, renches, kraines, ar, v...   

                                        title_tokens  \
0  [rand, ueen, lizabeth, rince, harles, rince, h...   
1       [ost, rump, id, reeze, renches, kraines, ar]   

                                     abstract_tokens  \
0  [hop, notebook, jacket, royal, cant, live, wit...   
1  [van, olchanets, peeked, parapet, sand, bag, f...   

                                          text_tfidf  \
0  rand ueen lizabeth rince harles rince hilip we...   
1  ost rump id reeze renches kraines ar van olcha...   

                                            text_w2v  \
0  [rand, ueen, lizabeth, rince, harles, rince, h...   
1  [ost, rump, id, reeze, renches, kraines, ar, v...   

                                           text_bert  \
0  The Brands Queen Elizabeth, Prince Charles, an...   
1  The Cost of Trump's Aid Freeze in the Trenches...   

                                   vector_cat_subcat  \
0  [0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, ...   
1  [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, ...   

                                        vector_tfidf  \
0    (0, 726)\t0.2356566551182428\n  (0, 1959)\t0...   
1    (0, 186)\t0.21543184635144275\n  (0, 321)\t0...   

                                          vector_w2v  
0  [0.047399126, -0.09589149, -0.13837357, 0.0836...  
1  [0.030284544, -0.030569091, -0.072350785, 0.10...

#### Get vectors/embeddings Title and Abstract using BERT

In [30]:
# TODO: load saved bert model
from google.colab import files
uploaded = files.upload()

Saving bert-base.zip to bert-base.zip


In [31]:
import zipfile
import os

# Unzip the uploaded model
zip_path = "bert-base.zip"
extract_path = "bert-base"

with zipfile.ZipFile(zip_path, 'r') as zip_ref:
    zip_ref.extractall(extract_path)

print("Extracted files:", os.listdir(extract_path))

Extracted files: ['bert-base', '__MACOSX']


In [32]:
import torch
from transformers import BertTokenizer, BertModel
from tqdm import tqdm

In [33]:
# TODO: Get BERT embeddings Title and Abstract using Hugging Face models and store vectors in a new column
model_path = "bert-base/bert-base"
tokenizer = BertTokenizer.from_pretrained(model_path)
model = BertModel.from_pretrained(model_path)
model.eval()
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

def get_bert_embedding(text):
    inputs = tokenizer(text, return_tensors='pt', truncation=True, padding=True, max_length=512)
    inputs = {key: val.to(device) for key, val in inputs.items()}

    with torch.no_grad():
        outputs = model(**inputs)

    cls_embedding = outputs.last_hidden_state[:, 0, :]
    return cls_embedding.squeeze().cpu().numpy()

bert_vectors = []
for text in tqdm(news_val_tokenized['text_bert'].astype(str).tolist(), desc="Encoding with BERT"):
    vec = get_bert_embedding(text)
    bert_vectors.append(vec)

news_val_tokenized['vector_bert'] = bert_vectors
print(news_val_tokenized[['news_id', 'vector_bert']].head(2))

Encoding with BERT: 100%|██████████| 40393/40393 [06:33<00:00, 102.56it/s]


  news_id                                        vector_bert
0  N55528  [0.03229502, 0.50512636, 0.13134134, 0.2930624...
1  N61837  [-0.27661568, 0.17622536, -0.35515082, 0.23503...


Above code cell took 7 minutes to run with colab's GPU. Without colab's GPU, it would have taken 2-3 hours.

In [34]:
print(news_val_tokenized.shape)
news_val_tokenized.head()

(40393, 19)


news_id   category      subcategory  \
0  N55528  lifestyle  lifestyleroyals   
1  N61837       news        newsworld   
2  N53526     health           voices   
3  N38324     health          medical   
4   N2073     sports     football_nfl   

                                               title  \
0  The Brands Queen Elizabeth, Prince Charles, an...   
1  The Cost of Trump's Aid Freeze in the Trenches...   
2  I Was An NBA Wife. Here's How It Affected My M...   
3  How to Get Rid of Skin Tags, According to a De...   
4  Should NFL be able to fine players for critici...   

                                            abstract  \
0  Shop the notebooks, jackets, and more that the...   
1  Lt. Ivan Molchanets peeked over a parapet of s...   
2  I felt like I was a fraud, and being an NBA wi...   
3  They seem harmless, but there's a very good re...   
4  Several fines came down against NFL players fo...   

                                             url  \
0  https://assets.msn.com/labs/mind/AAGH0ET.html   
1  https://assets.msn.com/labs/mind/AAJgNsz.html   
2  https://assets.msn.com/labs/mind/AACk2N6.html   
3  https://assets.msn.com/labs/mind/AAAKEkt.html   
4  https://assets.msn.com/labs/mind/AAJ4lap.html   

                                      title_entities  \
0  [{"Label": "Prince Philip, Duke of Edinburgh",...   
1                                                 []   
2                                                 []   
3  [{"Label": "Skin tag", "Type": "C", "WikidataI...   
4  [{"Label": "National Football League", "Type":...   

                                   abstract_entities  \
0                                                 []   
1  [{"Label": "Ukraine", "Type": "G", "WikidataId...   
2  [{"Label": "National Basketball Association", ...   
3  [{"Label": "Skin tag", "Type": "C", "WikidataI...   
4  [{"Label": "National Football League", "Type":...   

                                                text  \
0  The Brands Queen Elizabeth, Prince Charles, an...   
1  The Cost of Trump's Aid Freeze in the Trenches...   
2  I Was An NBA Wife. Here's How It Affected My M...   
3  How to Get Rid of Skin Tags, According to a De...   
4  Should NFL be able to fine players for critici...   

                                         text_tokens  \
0  [rand, ueen, lizabeth, rince, harles, rince, h...   
1  [ost, rump, id, reeze, renches, kraines, ar, v...   
2  [n, ife, eres, ow, ffected, ental, ealth, felt...   
3  [ow, et, id, kin, ag, ccording, ermatologist, ...   
4  [hould, able, fine, player, criticizing, offic...   

                                        title_tokens  \
0  [rand, ueen, lizabeth, rince, harles, rince, h...   
1       [ost, rump, id, reeze, renches, kraines, ar]   
2          [n, ife, eres, ow, ffected, ental, ealth]   
3      [ow, et, id, kin, ag, ccording, ermatologist]   
4  [hould, able, fine, player, criticizing, offic...   

                                     abstract_tokens  \
0  [hop, notebook, jacket, royal, cant, live, wit...   
1  [van, olchanets, peeked, parapet, sand, bag, f...   
2  [felt, like, fraud, wife, didnt, help, n, fact...   
3  [hey, seem, harmless, there, good, reason, sho...   
4  [everal, fine, came, player, criticizing, offi...   

                                          text_tfidf  \
0  rand ueen lizabeth rince harles rince hilip we...   
1  ost rump id reeze renches kraines ar van olcha...   
2  n ife eres ow ffected ental ealth felt like fr...   
3  ow et id kin ag ccording ermatologist hey seem...   
4  hould able fine player criticizing officiating...   

                                            text_w2v  \
0  [rand, ueen, lizabeth, rince, harles, rince, h...   
1  [ost, rump, id, reeze, renches, kraines, ar, v...   
2  [n, ife, eres, ow, ffected, ental, ealth, felt...   
3  [ow, et, id, kin, ag, ccording, ermatologist, ...   
4  [hould, able, fine, player, criticizing, offic...   

                                           text_bert  \
0  The Brands Qu

In [35]:
print(len(news_val_tokenized['vector_bert'][0]))
news_val_tokenized['vector_bert'][0]

768


array([ 3.22950184e-02,  5.05126357e-01,  1.31341338e-01,  2.93062478e-01,
       -5.57363451e-01, -3.66113633e-01,  1.62348747e-01,  5.57718277e-01,
       -3.74193907e-01, -3.54691714e-01, -1.38011441e-01,  1.48088947e-01,
        1.45582244e-01,  1.00607872e+00,  4.61494982e-01,  1.34784982e-01,
       -3.11519563e-01,  9.27519858e-01,  6.10260129e-01, -6.68620542e-02,
       -8.61717761e-02,  7.46763870e-02, -1.22379325e-01,  4.50138360e-01,
        5.93311489e-01,  3.04145575e-01,  1.15371764e-01, -4.33331460e-01,
       -2.23647788e-01,  4.67055351e-01, -3.27056020e-01,  3.03984255e-01,
       -3.07067186e-01, -5.61102986e-01,  3.99101406e-01, -3.37750971e-01,
        4.60502326e-01, -4.23770368e-01, -1.44565582e-01, -3.72148752e-01,
        1.32570267e-01,  1.04441740e-01,  2.88870186e-01, -1.97250903e-01,
       -1.16919927e-01, -5.91276646e-01, -3.29417801e+00,  3.00590485e-01,
        2.05668792e-01, -2.87197560e-01,  1.68726116e-01, -7.83745766e-01,
        9.87700075e-02,  

### Checkpoint: Save news_val_tokenized

In [36]:
print(type(news_val_tokenized))
print(news_val_tokenized.shape)
news_val_tokenized.head(2)

<class 'pandas.core.frame.DataFrame'>
(40393, 19)


news_id   category      subcategory  \
0  N55528  lifestyle  lifestyleroyals   
1  N61837       news        newsworld   

                                               title  \
0  The Brands Queen Elizabeth, Prince Charles, an...   
1  The Cost of Trump's Aid Freeze in the Trenches...   

                                            abstract  \
0  Shop the notebooks, jackets, and more that the...   
1  Lt. Ivan Molchanets peeked over a parapet of s...   

                                             url  \
0  https://assets.msn.com/labs/mind/AAGH0ET.html   
1  https://assets.msn.com/labs/mind/AAJgNsz.html   

                                      title_entities  \
0  [{"Label": "Prince Philip, Duke of Edinburgh",...   
1                                                 []   

                                   abstract_entities  \
0                                                 []   
1  [{"Label": "Ukraine", "Type": "G", "WikidataId...   

                                                text  \
0  The Brands Queen Elizabeth, Prince Charles, an...   
1  The Cost of Trump's Aid Freeze in the Trenches...   

                                         text_tokens  \
0  [rand, ueen, lizabeth, rince, harles, rince, h...   
1  [ost, rump, id, reeze, renches, kraines, ar, v...   

                                        title_tokens  \
0  [rand, ueen, lizabeth, rince, harles, rince, h...   
1       [ost, rump, id, reeze, renches, kraines, ar]   

                                     abstract_tokens  \
0  [hop, notebook, jacket, royal, cant, live, wit...   
1  [van, olchanets, peeked, parapet, sand, bag, f...   

                                          text_tfidf  \
0  rand ueen lizabeth rince harles rince hilip we...   
1  ost rump id reeze renches kraines ar van olcha...   

                                            text_w2v  \
0  [rand, ueen, lizabeth, rince, harles, rince, h...   
1  [ost, rump, id, reeze, renches, kraines, ar, v...   

                                           text_bert  \
0  The Brands Queen Elizabeth, Prince Charles, an...   
1  The Cost of Trump's Aid Freeze in the Trenches...   

                                   vector_cat_subcat  \
0  [0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, ...   
1  [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, ...   

                                        vector_tfidf  \
0    (0, 726)\t0.2356566551182428\n  (0, 1959)\t0...   
1    (0, 186)\t0.21543184635144275\n  (0, 321)\t0...   

                                          vector_w2v  \
0  [0.047399126, -0.09589149, -0.13837357, 0.0836...   
1  [0.030284544, -0.030569091, -0.072350785, 0.10...   

                                         vector_bert  
0  [0.03229502, 0.50512636, 0.13134134, 0.2930624...  
1  [-0.27661568, 0.17622536, -0.35515082, 0.23503...

In [37]:
print(news_val_tokenized['vector_tfidf'][0])
news_val_tokenized['vector_tfidf'][0].toarray()

  (0, 726)	0.2356566551182428
  (0, 1959)	0.2763176301512922
  (0, 2084)	0.2980356815605162
  (0, 2594)	0.2244142970661041
  (0, 2598)	0.25706190475590734
  (0, 3620)	0.2588589414063623
  (0, 3852)	0.5466155134989971
  (0, 3927)	0.28277919343931823
  (0, 4632)	0.2896856259993324
  (0, 4864)	0.29475349174752946
  (0, 4920)	0.21514812312551826


array([[0., 0., 0., ..., 0., 0., 0.]])

In [38]:
news_val_tokenized['vector_tfidf_dense'] = news_val_tokenized['vector_tfidf'].apply(
    lambda x: x.toarray().flatten() # Convert to dense and flatten to 1D array
)

In [39]:
# Drop the original sparse column
news_val_tokenized_to_save = news_val_tokenized.drop(columns=['vector_tfidf'])

In [40]:
print(type(news_val_tokenized_to_save))
print(news_val_tokenized_to_save.shape)
news_val_tokenized_to_save.head(2)

<class 'pandas.core.frame.DataFrame'>
(40393, 19)


news_id   category      subcategory  \
0  N55528  lifestyle  lifestyleroyals   
1  N61837       news        newsworld   

                                               title  \
0  The Brands Queen Elizabeth, Prince Charles, an...   
1  The Cost of Trump's Aid Freeze in the Trenches...   

                                            abstract  \
0  Shop the notebooks, jackets, and more that the...   
1  Lt. Ivan Molchanets peeked over a parapet of s...   

                                             url  \
0  https://assets.msn.com/labs/mind/AAGH0ET.html   
1  https://assets.msn.com/labs/mind/AAJgNsz.html   

                                      title_entities  \
0  [{"Label": "Prince Philip, Duke of Edinburgh",...   
1                                                 []   

                                   abstract_entities  \
0                                                 []   
1  [{"Label": "Ukraine", "Type": "G", "WikidataId...   

                                                text  \
0  The Brands Queen Elizabeth, Prince Charles, an...   
1  The Cost of Trump's Aid Freeze in the Trenches...   

                                         text_tokens  \
0  [rand, ueen, lizabeth, rince, harles, rince, h...   
1  [ost, rump, id, reeze, renches, kraines, ar, v...   

                                        title_tokens  \
0  [rand, ueen, lizabeth, rince, harles, rince, h...   
1       [ost, rump, id, reeze, renches, kraines, ar]   

                                     abstract_tokens  \
0  [hop, notebook, jacket, royal, cant, live, wit...   
1  [van, olchanets, peeked, parapet, sand, bag, f...   

                                          text_tfidf  \
0  rand ueen lizabeth rince harles rince hilip we...   
1  ost rump id reeze renches kraines ar van olcha...   

                                            text_w2v  \
0  [rand, ueen, lizabeth, rince, harles, rince, h...   
1  [ost, rump, id, reeze, renches, kraines, ar, v...   

                                           text_bert  \
0  The Brands Queen Elizabeth, Prince Charles, an...   
1  The Cost of Trump's Aid Freeze in the Trenches...   

                                   vector_cat_subcat  \
0  [0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, ...   
1  [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, ...   

                                          vector_w2v  \
0  [0.047399126, -0.09589149, -0.13837357, 0.0836...   
1  [0.030284544, -0.030569091, -0.072350785, 0.10...   

                                         vector_bert  \
0  [0.03229502, 0.50512636, 0.13134134, 0.2930624...   
1  [-0.27661568, 0.17622536, -0.35515082, 0.23503...   

                                  vector_tfidf_dense  
0  [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...  
1  [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...

In [41]:
# Define the filename for the downloaded parquet
filename = 'news_val_with_embeddings_dense.parquet'

# Save the DataFrame to a temporary parquet file in Colab's environment
news_val_tokenized_to_save.to_parquet(filename, index=False)

# Trigger the download
try:
    files.download(filename)
    print(f"DataFrame downloaded as: {filename}")
except:
    pass

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

DataFrame downloaded as: news_val_with_embeddings_dense.parquet


### Load news_train_with_embeddings_dense.parquet

In [1]:
import numpy as np
import pandas as pd

In [2]:
# REMINDER: This is use to load the data back to the original format from parquet files
try:
    news_val_tokenized_to_save = pd.read_parquet('MINDsmall_valprocessed/news_val_with_embeddings_dense.parquet')
except FileNotFoundError:
    from google.colab import files
    files.upload()
    news_val_tokenized_to_save = pd.read_parquet('news_val_with_embeddings_dense.parquet')

Saving news_val_with_embeddings_dense.parquet to news_val_with_embeddings_dense.parquet


In [3]:
print(type(news_val_tokenized_to_save))
print(news_val_tokenized_to_save.shape)
news_val_tokenized_to_save.head()

<class 'pandas.core.frame.DataFrame'>
(40393, 19)


news_id   category      subcategory  \
0  N55528  lifestyle  lifestyleroyals   
1  N61837       news        newsworld   
2  N53526     health           voices   
3  N38324     health          medical   
4   N2073     sports     football_nfl   

                                               title  \
0  The Brands Queen Elizabeth, Prince Charles, an...   
1  The Cost of Trump's Aid Freeze in the Trenches...   
2  I Was An NBA Wife. Here's How It Affected My M...   
3  How to Get Rid of Skin Tags, According to a De...   
4  Should NFL be able to fine players for critici...   

                                            abstract  \
0  Shop the notebooks, jackets, and more that the...   
1  Lt. Ivan Molchanets peeked over a parapet of s...   
2  I felt like I was a fraud, and being an NBA wi...   
3  They seem harmless, but there's a very good re...   
4  Several fines came down against NFL players fo...   

                                             url  \
0  https://assets.msn.com/labs/mind/AAGH0ET.html   
1  https://assets.msn.com/labs/mind/AAJgNsz.html   
2  https://assets.msn.com/labs/mind/AACk2N6.html   
3  https://assets.msn.com/labs/mind/AAAKEkt.html   
4  https://assets.msn.com/labs/mind/AAJ4lap.html   

                                      title_entities  \
0  [{"Label": "Prince Philip, Duke of Edinburgh",...   
1                                                 []   
2                                                 []   
3  [{"Label": "Skin tag", "Type": "C", "WikidataI...   
4  [{"Label": "National Football League", "Type":...   

                                   abstract_entities  \
0                                                 []   
1  [{"Label": "Ukraine", "Type": "G", "WikidataId...   
2  [{"Label": "National Basketball Association", ...   
3  [{"Label": "Skin tag", "Type": "C", "WikidataI...   
4  [{"Label": "National Football League", "Type":...   

                                                text  \
0  The Brands Queen Elizabeth, Prince Charles, an...   
1  The Cost of Trump's Aid Freeze in the Trenches...   
2  I Was An NBA Wife. Here's How It Affected My M...   
3  How to Get Rid of Skin Tags, According to a De...   
4  Should NFL be able to fine players for critici...   

                                         text_tokens  \
0  [rand, ueen, lizabeth, rince, harles, rince, h...   
1  [ost, rump, id, reeze, renches, kraines, ar, v...   
2  [n, ife, eres, ow, ffected, ental, ealth, felt...   
3  [ow, et, id, kin, ag, ccording, ermatologist, ...   
4  [hould, able, fine, player, criticizing, offic...   

                                        title_tokens  \
0  [rand, ueen, lizabeth, rince, harles, rince, h...   
1       [ost, rump, id, reeze, renches, kraines, ar]   
2          [n, ife, eres, ow, ffected, ental, ealth]   
3      [ow, et, id, kin, ag, ccording, ermatologist]   
4  [hould, able, fine, player, criticizing, offic...   

                                     abstract_tokens  \
0  [hop, notebook, jacket, royal, cant, live, wit...   
1  [van, olchanets, peeked, parapet, sand, bag, f...   
2  [felt, like, fraud, wife, didnt, help, n, fact...   
3  [hey, seem, harmless, there, good, reason, sho...   
4  [everal, fine, came, player, criticizing, offi...   

                                          text_tfidf  \
0  rand ueen lizabeth rince harles rince hilip we...   
1  ost rump id reeze renches kraines ar van olcha...   
2  n ife eres ow ffected ental ealth felt like fr...   
3  ow et id kin ag ccording ermatologist hey seem...   
4  hould able fine player criticizing officiating...   

                                            text_w2v  \
0  [rand, ueen, lizabeth, rince, harles, rince, h...   
1  [ost, rump, id, reeze, renches, kraines, ar, v...   
2  [n, ife, eres, ow, ffected, ental, ealth, felt...   
3  [ow, et, id, kin, ag, ccording, ermatologist, ...   
4  [hould, able, fine, player, criticizing, offic...   

                                           text_bert  \
0  The Brands Qu

### Define a function for normalization/scaling to all generated vectors (binary, TF-IDF, Word2Vec, BERT) before concatenation

In [4]:
# TODO:
def normalize_vector(v):
    v = np.array(v)
    norm = np.linalg.norm(v)
    return v / norm if norm > 0 else v

### Concatenate all processed vectors (Category, Subcategory, Title, Abstract) to form final item profile vectors. Because we want to compare the performance of using TFIDF, word2vec, and BERT, we would need 3 different representations of item vectors:

In [5]:
# For each row, normalize and concatenate
def concat_tfidf(row):
    return np.concatenate([
        normalize_vector(row["vector_cat_subcat"]),
        normalize_vector(row["vector_tfidf_dense"])
    ])

def concat_w2v(row):
    return np.concatenate([
        normalize_vector(row["vector_cat_subcat"]),
        normalize_vector(row["vector_w2v"])
    ])

def concat_bert(row):
    return np.concatenate([
        normalize_vector(row["vector_cat_subcat"]),
        normalize_vector(row["vector_bert"])
    ])

In [6]:
# TODO: store the final item profile vector in a new column
news_val_tokenized_to_save["item_vector_tfidf_dense"] = news_val_tokenized_to_save.apply(concat_tfidf, axis=1)
news_val_tokenized_to_save["item_vector_w2v"]   = news_val_tokenized_to_save.apply(concat_w2v, axis=1)
news_val_tokenized_to_save["item_vector_bert"]  = news_val_tokenized_to_save.apply(concat_bert, axis=1)

In [7]:
print(type(news_val_tokenized_to_save))
print(news_val_tokenized_to_save.shape)
news_val_tokenized_to_save.head()

<class 'pandas.core.frame.DataFrame'>
(40393, 22)


news_id   category      subcategory  \
0  N55528  lifestyle  lifestyleroyals   
1  N61837       news        newsworld   
2  N53526     health           voices   
3  N38324     health          medical   
4   N2073     sports     football_nfl   

                                               title  \
0  The Brands Queen Elizabeth, Prince Charles, an...   
1  The Cost of Trump's Aid Freeze in the Trenches...   
2  I Was An NBA Wife. Here's How It Affected My M...   
3  How to Get Rid of Skin Tags, According to a De...   
4  Should NFL be able to fine players for critici...   

                                            abstract  \
0  Shop the notebooks, jackets, and more that the...   
1  Lt. Ivan Molchanets peeked over a parapet of s...   
2  I felt like I was a fraud, and being an NBA wi...   
3  They seem harmless, but there's a very good re...   
4  Several fines came down against NFL players fo...   

                                             url  \
0  https://assets.msn.com/labs/mind/AAGH0ET.html   
1  https://assets.msn.com/labs/mind/AAJgNsz.html   
2  https://assets.msn.com/labs/mind/AACk2N6.html   
3  https://assets.msn.com/labs/mind/AAAKEkt.html   
4  https://assets.msn.com/labs/mind/AAJ4lap.html   

                                      title_entities  \
0  [{"Label": "Prince Philip, Duke of Edinburgh",...   
1                                                 []   
2                                                 []   
3  [{"Label": "Skin tag", "Type": "C", "WikidataI...   
4  [{"Label": "National Football League", "Type":...   

                                   abstract_entities  \
0                                                 []   
1  [{"Label": "Ukraine", "Type": "G", "WikidataId...   
2  [{"Label": "National Basketball Association", ...   
3  [{"Label": "Skin tag", "Type": "C", "WikidataI...   
4  [{"Label": "National Football League", "Type":...   

                                                text  \
0  The Brands Queen Elizabeth, Prince Charles, an...   
1  The Cost of Trump's Aid Freeze in the Trenches...   
2  I Was An NBA Wife. Here's How It Affected My M...   
3  How to Get Rid of Skin Tags, According to a De...   
4  Should NFL be able to fine players for critici...   

                                         text_tokens  ...  \
0  [rand, ueen, lizabeth, rince, harles, rince, h...  ...   
1  [ost, rump, id, reeze, renches, kraines, ar, v...  ...   
2  [n, ife, eres, ow, ffected, ental, ealth, felt...  ...   
3  [ow, et, id, kin, ag, ccording, ermatologist, ...  ...   
4  [hould, able, fine, player, criticizing, offic...  ...   

                                          text_tfidf  \
0  rand ueen lizabeth rince harles rince hilip we...   
1  ost rump id reeze renches kraines ar van olcha...   
2  n ife eres ow ffected ental ealth felt like fr...   
3  ow et id kin ag ccording ermatologist hey seem...   
4  hould able fine player criticizing officiating...   

                                            text_w2v  \
0  [rand, ueen, lizabeth, rince, harles, rince, h...   
1  [ost, rump, id, reeze, renches, kraines, ar, v...   
2  [n, ife, eres, ow, ffected, ental, ealth, felt...   
3  [ow, et, id, kin, ag, ccording, ermatologist, ...   
4  [hould, able, fine, player, criticizing, offic...   

                                           text_bert  \
0  The Brands Queen Elizabeth, Prince Charles, an...   
1  The Cost of Trump's Aid Freeze in the Trenches...   
2  I Was An NBA Wife. Here's How It Affected My M...   
3  How to Get Rid of Skin Tags, According to a De...   
4  Should NFL be able to fine players for critici...   

                                   vector_cat_subcat  \
0  [0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, ...   
1  [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, ...   
2  [0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...   
3  [0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...   
4  [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, ...   

                                          

In [11]:
print(len(news_val_tokenized_to_save['item_vector_tfidf_dense'][0]))
news_val_tokenized_to_save['item_vector_tfidf_dense'][0]

5267


array([0., 0., 0., ..., 0., 0., 0.])

In [12]:
print(len(news_val_tokenized_to_save['item_vector_w2v'][0]))
news_val_tokenized_to_save['item_vector_w2v'][0]

367


array([ 0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
        0.        ,  0.        ,  0.70710678,  0.        ,  0.        ,
        0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
        0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
        0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
        0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
        0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
        0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
        0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
        0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
        0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
        0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
        0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
        0.        ,  0.        ,  0.        ,  0.        ,  0.  

In [13]:
print(len(news_val_tokenized_to_save['item_vector_bert'][0]))
news_val_tokenized_to_save['item_vector_bert'][0]

1035


array([ 0.        ,  0.        ,  0.        , ..., -0.03417296,
        0.03808109,  0.02280931])

### Clean news_val_tokenized_to_save by making the file smaller (get rid of unnecessary columns)

In [14]:
# TODO: remove columns 'text', 'text_tokens', 'title_tokens', 'abstract_tokens', 'text_tfidf', 'text_w2v', 'text_bert',	'vector_cat_subcat', 'vector_w2v', 'vector_bert', 'vector_tfidf_dense'
remove_columns = [
    'text',
    'text_tokens',
    'title_tokens',
    'abstract_tokens',
    'text_tfidf',
    'text_w2v',
    'text_bert',
    'vector_cat_subcat',
    'vector_w2v',
    'vector_bert',
    'vector_tfidf_dense'
]
news_val_tokenized_to_save_final = news_val_tokenized_to_save.drop(columns=remove_columns)

In [15]:
print(type(news_val_tokenized_to_save_final))
print(news_val_tokenized_to_save_final.shape)
news_val_tokenized_to_save_final.head()

<class 'pandas.core.frame.DataFrame'>
(40393, 11)


news_id   category      subcategory  \
0  N55528  lifestyle  lifestyleroyals   
1  N61837       news        newsworld   
2  N53526     health           voices   
3  N38324     health          medical   
4   N2073     sports     football_nfl   

                                               title  \
0  The Brands Queen Elizabeth, Prince Charles, an...   
1  The Cost of Trump's Aid Freeze in the Trenches...   
2  I Was An NBA Wife. Here's How It Affected My M...   
3  How to Get Rid of Skin Tags, According to a De...   
4  Should NFL be able to fine players for critici...   

                                            abstract  \
0  Shop the notebooks, jackets, and more that the...   
1  Lt. Ivan Molchanets peeked over a parapet of s...   
2  I felt like I was a fraud, and being an NBA wi...   
3  They seem harmless, but there's a very good re...   
4  Several fines came down against NFL players fo...   

                                             url  \
0  https://assets.msn.com/labs/mind/AAGH0ET.html   
1  https://assets.msn.com/labs/mind/AAJgNsz.html   
2  https://assets.msn.com/labs/mind/AACk2N6.html   
3  https://assets.msn.com/labs/mind/AAAKEkt.html   
4  https://assets.msn.com/labs/mind/AAJ4lap.html   

                                      title_entities  \
0  [{"Label": "Prince Philip, Duke of Edinburgh",...   
1                                                 []   
2                                                 []   
3  [{"Label": "Skin tag", "Type": "C", "WikidataI...   
4  [{"Label": "National Football League", "Type":...   

                                   abstract_entities  \
0                                                 []   
1  [{"Label": "Ukraine", "Type": "G", "WikidataId...   
2  [{"Label": "National Basketball Association", ...   
3  [{"Label": "Skin tag", "Type": "C", "WikidataI...   
4  [{"Label": "National Football League", "Type":...   

                             item_vector_tfidf_dense  \
0  [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.70710678...   
1  [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...   
2  [0.0, 0.0, 0.0, 0.0, 0.0, 0.7071067811865475, ...   
3  [0.0, 0.0, 0.0, 0.0, 0.0, 0.7071067811865475, ...   
4  [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...   

                                     item_vector_w2v  \
0  [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.70710678...   
1  [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...   
2  [0.0, 0.0, 0.0, 0.0, 0.0, 0.7071067811865475, ...   
3  [0.0, 0.0, 0.0, 0.0, 0.0, 0.7071067811865475, ...   
4  [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...   

                                    item_vector_bert  
0  [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.70710678...  
1  [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...  
2  [0.0, 0.0, 0.0, 0.0, 0.0, 0.7071067811865475, ...  
3  [0.0, 0.0, 0.0, 0.0, 0.0, 0.7071067811865475, ...  
4  [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...

In [16]:
print(len(news_val_tokenized_to_save_final['item_vector_tfidf_dense'][0]))
news_val_tokenized_to_save_final['item_vector_tfidf_dense'][0]

5267


array([0., 0., 0., ..., 0., 0., 0.])

In [17]:
print(len(news_val_tokenized_to_save_final['item_vector_w2v'][0]))
news_val_tokenized_to_save_final['item_vector_w2v'][0]

367


array([ 0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
        0.        ,  0.        ,  0.70710678,  0.        ,  0.        ,
        0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
        0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
        0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
        0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
        0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
        0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
        0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
        0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
        0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
        0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
        0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
        0.        ,  0.        ,  0.        ,  0.        ,  0.  

In [18]:
print(len(news_val_tokenized_to_save_final['item_vector_bert'][0]))
news_val_tokenized_to_save_final['item_vector_bert'][0]

1035


array([ 0.        ,  0.        ,  0.        , ..., -0.03417296,
        0.03808109,  0.02280931])

### Checkpoint: Save item vectors

In [19]:
# Define the filename for the downloaded parquet
filename = 'news_val_with_item_vectors.parquet'

# Save the DataFrame to a temporary parquet file in Colab's environment
news_val_tokenized_to_save_final.to_parquet(filename, index=False)

# Trigger the download
try:
    files.download(filename)
    print(f"DataFrame downloaded as: {filename}")
except:
    pass

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

DataFrame downloaded as: news_val_with_item_vectors.parquet


### Build User Profile

In [1]:
import numpy as np
import pandas as pd

In [2]:
# REMINDER: This is use to load the data back to the original format from parquet files
try:
    behaviors_val_preprocessed = pd.read_parquet('MINDsmall_val_processed/behaviors_val_preprocessed.parquet')
except FileNotFoundError:
    from google.colab import files
    files.upload()
    behaviors_val_preprocessed = pd.read_parquet('behaviors_val_preprocessed.parquet')

Saving behaviors_val_preprocessed.parquet to behaviors_val_preprocessed.parquet


In [3]:
print(type(behaviors_val_preprocessed))
print(behaviors_val_preprocessed.shape)
behaviors_val_preprocessed.head()

<class 'pandas.core.frame.DataFrame'>
(2670611, 6)


impression_id user_id                time  \
0              1  U80234 2019-11-15 12:37:50   
0              1  U80234 2019-11-15 12:37:50   
0              1  U80234 2019-11-15 12:37:50   
0              1  U80234 2019-11-15 12:37:50   
0              1  U80234 2019-11-15 12:37:50   

                                             history candidate_news_id  \
0  [N55189, N46039, N51741, N53234, N11276, N264,...            N28682   
0  [N55189, N46039, N51741, N53234, N11276, N264,...            N48740   
0  [N55189, N46039, N51741, N53234, N11276, N264,...            N31958   
0  [N55189, N46039, N51741, N53234, N11276, N264,...            N34130   
0  [N55189, N46039, N51741, N53234, N11276, N264,...             N6916   

   clicked  
0        0  
0        0  
0        1  
0        0  
0        0

In [4]:
# REMINDER: This is use to load the data back to the original format from parquet files
try:
    item_profile = pd.read_parquet('MINDsmall_val_processed/news_val_with_item_vectors.parquet')
except FileNotFoundError:
    from google.colab import files
    files.upload()
    item_profile = pd.read_parquet('news_val_with_item_vectors.parquet')

Saving news_val_with_item_vectors.parquet to news_val_with_item_vectors.parquet


In [5]:
print(type(item_profile))
print(item_profile.shape)
item_profile.head()

<class 'pandas.core.frame.DataFrame'>
(40393, 11)


news_id   category      subcategory  \
0  N55528  lifestyle  lifestyleroyals   
1  N61837       news        newsworld   
2  N53526     health           voices   
3  N38324     health          medical   
4   N2073     sports     football_nfl   

                                               title  \
0  The Brands Queen Elizabeth, Prince Charles, an...   
1  The Cost of Trump's Aid Freeze in the Trenches...   
2  I Was An NBA Wife. Here's How It Affected My M...   
3  How to Get Rid of Skin Tags, According to a De...   
4  Should NFL be able to fine players for critici...   

                                            abstract  \
0  Shop the notebooks, jackets, and more that the...   
1  Lt. Ivan Molchanets peeked over a parapet of s...   
2  I felt like I was a fraud, and being an NBA wi...   
3  They seem harmless, but there's a very good re...   
4  Several fines came down against NFL players fo...   

                                             url  \
0  https://assets.msn.com/labs/mind/AAGH0ET.html   
1  https://assets.msn.com/labs/mind/AAJgNsz.html   
2  https://assets.msn.com/labs/mind/AACk2N6.html   
3  https://assets.msn.com/labs/mind/AAAKEkt.html   
4  https://assets.msn.com/labs/mind/AAJ4lap.html   

                                      title_entities  \
0  [{"Label": "Prince Philip, Duke of Edinburgh",...   
1                                                 []   
2                                                 []   
3  [{"Label": "Skin tag", "Type": "C", "WikidataI...   
4  [{"Label": "National Football League", "Type":...   

                                   abstract_entities  \
0                                                 []   
1  [{"Label": "Ukraine", "Type": "G", "WikidataId...   
2  [{"Label": "National Basketball Association", ...   
3  [{"Label": "Skin tag", "Type": "C", "WikidataI...   
4  [{"Label": "National Football League", "Type":...   

                             item_vector_tfidf_dense  \
0  [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.70710678...   
1  [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...   
2  [0.0, 0.0, 0.0, 0.0, 0.0, 0.7071067811865475, ...   
3  [0.0, 0.0, 0.0, 0.0, 0.0, 0.7071067811865475, ...   
4  [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...   

                                     item_vector_w2v  \
0  [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.70710678...   
1  [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...   
2  [0.0, 0.0, 0.0, 0.0, 0.0, 0.7071067811865475, ...   
3  [0.0, 0.0, 0.0, 0.0, 0.0, 0.7071067811865475, ...   
4  [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...   

                                    item_vector_bert  
0  [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.70710678...  
1  [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...  
2  [0.0, 0.0, 0.0, 0.0, 0.0, 0.7071067811865475, ...  
3  [0.0, 0.0, 0.0, 0.0, 0.0, 0.7071067811865475, ...  
4  [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...

In [6]:
# Unexplode behaviors_val_preprocessed
behaviors_val_preprocessed = behaviors_val_preprocessed.groupby(
    ['impression_id', 'user_id', 'time']
  ).agg({
     'candidate_news_id': list,
     'clicked': list,
     'history': 'first'
  }).reset_index()

behaviors_val_preprocessed['impressions'] = behaviors_val_preprocessed.apply(
  lambda row: list(zip(row['candidate_news_id'], row['clicked'])),
  axis=1
)

In [7]:
print(behaviors_val_preprocessed.shape)
behaviors_val_preprocessed.head()

(73152, 7)


impression_id user_id                time  \
0              1  U80234 2019-11-15 12:37:50   
1              2  U60458 2019-11-15 07:11:50   
2              3  U44190 2019-11-15 09:55:12   
3              4  U87380 2019-11-15 15:12:46   
4              5   U9444 2019-11-15 08:25:46   

                                   candidate_news_id  \
0  [N28682, N48740, N31958, N34130, N6916, N5472,...   
1   [N20036, N32536, N46976, N35216, N36779, N31958]   
2  [N36779, N62365, N58098, N5472, N13408, N55036...   
3  [N6950, N60215, N6074, N11930, N6916, N24802, ...   
4  [N5940, N49285, N23355, N19990, N31958, N29393...   

                                             clicked  \
0  [0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...   
1                                 [0, 0, 0, 0, 0, 0]   
2  [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, ...   
3  [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, ...   
4            [1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0]   

                                             history  \
0  [N55189, N46039, N51741, N53234, N11276, N264,...   
1  [N58715, N32109, N51180, N33438, N54827, N2848...   
2  [N56253, N1150, N55189, N16233, N61704, N51706...   
3  [N63554, N49153, N28678, N43369, N58518, N4440...   
4           [N51692, N18285, N26015, N22679, N55556]   

                                         impressions  
0  [(N28682, 0), (N48740, 0), (N31958, 1), (N3413...  
1  [(N20036, 0), (N32536, 0), (N46976, 0), (N3521...  
2  [(N36779, 0), (N62365, 0), (N58098, 0), (N5472...  
3  [(N6950, 0), (N60215, 0), (N6074, 0), (N11930,...  
4  [(N5940, 1), (N49285, 0), (N23355, 0), (N19990...

In [8]:
# Create dictionaries for quick lookup
tfidf_dict = dict(zip(item_profile["news_id"], item_profile["item_vector_tfidf_dense"]))
w2v_dict   = dict(zip(item_profile["news_id"], item_profile["item_vector_w2v"]))
bert_dict  = dict(zip(item_profile["news_id"], item_profile["item_vector_bert"]))

In [9]:
# Define Averaging function
def average_vectors(news_ids, item_vector_dict):
    vectors = [item_vector_dict[nid] for nid in news_ids if nid in item_vector_dict]
    return np.mean(vectors, axis=0) if vectors else np.zeros(len(next(iter(item_vector_dict.values()))))

In [10]:
# Get user -> history mapping
user_histories = (
    behaviors_val_preprocessed.groupby("user_id")["history"]
    .first()
    .to_dict()
)

In [11]:
# Build user profile vectors (one per user)
user_profiles_tfidf = {}
user_profiles_w2v = {}
user_profiles_bert = {}

for user_id, history in user_histories.items():
    user_profiles_tfidf[user_id] = average_vectors(history, tfidf_dict)
    user_profiles_w2v[user_id]   = average_vectors(history, w2v_dict)
    user_profiles_bert[user_id]  = average_vectors(history, bert_dict)

In [12]:
# Create a user profiles DataFrame
user_profile_df = pd.DataFrame({
    "user_id": list(user_profiles_tfidf.keys()),
    "user_profile_vector_tfidf_avg": list(user_profiles_tfidf.values()),
    "user_profile_vector_w2v_avg": list(user_profiles_w2v.values()),
    "user_profile_vector_bert_avg": list(user_profiles_bert.values())
})

In [13]:
# Merge into behaviors_val_preprocessed
behaviors_val_preprocessed = behaviors_val_preprocessed.merge(
    user_profile_df, on="user_id", how="left"
)

In [14]:
print(behaviors_val_preprocessed.shape)
behaviors_val_preprocessed.head()

(73152, 10)


impression_id user_id                time  \
0              1  U80234 2019-11-15 12:37:50   
1              2  U60458 2019-11-15 07:11:50   
2              3  U44190 2019-11-15 09:55:12   
3              4  U87380 2019-11-15 15:12:46   
4              5   U9444 2019-11-15 08:25:46   

                                   candidate_news_id  \
0  [N28682, N48740, N31958, N34130, N6916, N5472,...   
1   [N20036, N32536, N46976, N35216, N36779, N31958]   
2  [N36779, N62365, N58098, N5472, N13408, N55036...   
3  [N6950, N60215, N6074, N11930, N6916, N24802, ...   
4  [N5940, N49285, N23355, N19990, N31958, N29393...   

                                             clicked  \
0  [0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...   
1                                 [0, 0, 0, 0, 0, 0]   
2  [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, ...   
3  [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, ...   
4            [1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0]   

                                             history  \
0  [N55189, N46039, N51741, N53234, N11276, N264,...   
1  [N58715, N32109, N51180, N33438, N54827, N2848...   
2  [N56253, N1150, N55189, N16233, N61704, N51706...   
3  [N63554, N49153, N28678, N43369, N58518, N4440...   
4           [N51692, N18285, N26015, N22679, N55556]   

                                         impressions  \
0  [(N28682, 0), (N48740, 0), (N31958, 1), (N3413...   
1  [(N20036, 0), (N32536, 0), (N46976, 0), (N3521...   
2  [(N36779, 0), (N62365, 0), (N58098, 0), (N5472...   
3  [(N6950, 0), (N60215, 0), (N6074, 0), (N11930,...   
4  [(N5940, 1), (N49285, 0), (N23355, 0), (N19990...   

                       user_profile_vector_tfidf_avg  \
0  [0.04714045207910316, 0.04714045207910316, 0.0...   
1  [0.0, 0.0, 0.21757131728816845, 0.0, 0.0, 0.05...   
2  [0.0, 0.07856742013183861, 0.0, 0.0, 0.0, 0.0,...   
3  [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...   
4  [0.0, 0.1414213562373095, 0.1414213562373095, ...   

                         user_profile_vector_w2v_avg  \
0  [0.04714045207910316, 0.04714045207910316, 0.0...   
1  [0.0, 0.0, 0.21757131728816845, 0.0, 0.0, 0.05...   
2  [0.0, 0.07856742013183861, 0.0, 0.0, 0.0, 0.0,...   
3  [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...   
4  [0.0, 0.1414213562373095, 0.1414213562373095, ...   

                        user_profile_vector_bert_avg  
0  [0.04714045207910316, 0.04714045207910316, 0.0...  
1  [0.0, 0.0, 0.21757131728816845, 0.0, 0.0, 0.05...  
2  [0.0, 0.07856742013183861, 0.0, 0.0, 0.0, 0.0,...  
3  [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...  
4  [0.0, 0.1414213562373095, 0.1414213562373095, ...

In [15]:
behaviors_val_preprocessed['user_profile_vector_tfidf_avg'][0]

array([0.04714045, 0.04714045, 0.04714045, ..., 0.        , 0.        ,
       0.        ])

In [16]:
behaviors_val_preprocessed['user_profile_vector_w2v_avg'][0]

array([ 0.04714045,  0.04714045,  0.04714045,  0.        ,  0.        ,
        0.04714045,  0.        ,  0.04714045,  0.        ,  0.04714045,
        0.        ,  0.23570226,  0.        ,  0.        ,  0.18856181,
        0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
        0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
        0.04714045,  0.        ,  0.        ,  0.        ,  0.        ,
        0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
        0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
        0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
        0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
        0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
        0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
        0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
        0.        ,  0.        ,  0.        ,  0.        ,  0.  

In [17]:
behaviors_val_preprocessed['user_profile_vector_bert_avg'][0]

array([ 0.04714045,  0.04714045,  0.04714045, ..., -0.02284816,
        0.0226607 ,  0.0159667 ])

### Checkpoint: save user profile

In [18]:
# 1. DataFrame with tf-idf vector
df_tfidf = behaviors_val_preprocessed[[
    'impression_id', 'user_id', 'time', 'candidate_news_id',
    'clicked', 'history', 'impressions', 'user_profile_vector_tfidf_avg'
]]

# 2. DataFrame with word2vec vector
df_w2v = behaviors_val_preprocessed[[
    'impression_id', 'user_id', 'time', 'candidate_news_id',
    'clicked', 'history', 'impressions', 'user_profile_vector_w2v_avg'
]]

# 3. DataFrame with BERT vector
df_bert = behaviors_val_preprocessed[[
    'impression_id', 'user_id', 'time', 'candidate_news_id',
    'clicked', 'history', 'impressions', 'user_profile_vector_bert_avg'
]]

In [19]:
print(df_tfidf.shape)
df_tfidf.head(1)

(73152, 8)


impression_id user_id                time  \
0              1  U80234 2019-11-15 12:37:50   

                                   candidate_news_id  \
0  [N28682, N48740, N31958, N34130, N6916, N5472,...   

                                             clicked  \
0  [0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...   

                                             history  \
0  [N55189, N46039, N51741, N53234, N11276, N264,...   

                                         impressions  \
0  [(N28682, 0), (N48740, 0), (N31958, 1), (N3413...   

                       user_profile_vector_tfidf_avg  
0  [0.04714045207910316, 0.04714045207910316, 0.0...

In [20]:
print(df_w2v.shape)
df_w2v.head(1)

(73152, 8)


impression_id user_id                time  \
0              1  U80234 2019-11-15 12:37:50   

                                   candidate_news_id  \
0  [N28682, N48740, N31958, N34130, N6916, N5472,...   

                                             clicked  \
0  [0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...   

                                             history  \
0  [N55189, N46039, N51741, N53234, N11276, N264,...   

                                         impressions  \
0  [(N28682, 0), (N48740, 0), (N31958, 1), (N3413...   

                         user_profile_vector_w2v_avg  
0  [0.04714045207910316, 0.04714045207910316, 0.0...

In [21]:
print(df_bert.shape)
df_bert.head(1)

(73152, 8)


impression_id user_id                time  \
0              1  U80234 2019-11-15 12:37:50   

                                   candidate_news_id  \
0  [N28682, N48740, N31958, N34130, N6916, N5472,...   

                                             clicked  \
0  [0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...   

                                             history  \
0  [N55189, N46039, N51741, N53234, N11276, N264,...   

                                         impressions  \
0  [(N28682, 0), (N48740, 0), (N31958, 1), (N3413...   

                        user_profile_vector_bert_avg  
0  [0.04714045207910316, 0.04714045207910316, 0.0...

In [22]:
df_tfidf.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 73152 entries, 0 to 73151
Data columns (total 8 columns):
 #   Column                         Non-Null Count  Dtype         
---  ------                         --------------  -----         
 0   impression_id                  73152 non-null  int64         
 1   user_id                        73152 non-null  object        
 2   time                           73152 non-null  datetime64[ns]
 3   candidate_news_id              73152 non-null  object        
 4   clicked                        73152 non-null  object        
 5   history                        73152 non-null  object        
 6   impressions                    73152 non-null  object        
 7   user_profile_vector_tfidf_avg  73152 non-null  object        
dtypes: datetime64[ns](1), int64(1), object(6)
memory usage: 4.5+ MB


In [23]:
print(len(df_tfidf["user_profile_vector_tfidf_avg"][0]))
print(df_tfidf["user_profile_vector_tfidf_avg"].iloc[0])
print(type(df_tfidf["user_profile_vector_tfidf_avg"].iloc[0]))
print(type(df_tfidf["user_profile_vector_tfidf_avg"].iloc[0][0]))

5267
[0.04714045 0.04714045 0.04714045 ... 0.         0.         0.        ]
<class 'numpy.ndarray'>
<class 'numpy.float64'>


In [ ]:
# Define the filename for the downloaded parquet
filename = 'behaviors_val_user_profile_tfidf.parquet'

# Save the DataFrame to a temporary parquet file in Colab's environment
df_tfidf.to_parquet(filename, index=False)

# Trigger the download
try:
    files.download(filename)
    print(f"DataFrame downloaded as: {filename}")
except:
    pass

In [ ]:
# Define the filename for the downloaded parquet
filename = 'behaviors_val_user_profile_w2v.parquet'

# Save the DataFrame to a temporary parquet file in Colab's environment
df_w2v.to_parquet(filename, index=False)

# Trigger the download
try:
    files.download(filename)
    print(f"DataFrame downloaded as: {filename}")
except:
    pass

In [ ]:
# Define the filename for the downloaded parquet
filename = 'behaviors_val_user_profile_bert.parquet'

# Save the DataFrame to a temporary parquet file in Colab's environment
df_bert.to_parquet(filename, index=False)

# Trigger the download
try:
    files.download(filename)
    print(f"DataFrame downloaded as: {filename}")
except:
    pass

### Debug

In [ ]:
# REMINDER: This is use to load the data back to the original format from parquet files
try:
    news_train_tokenized = pd.read_parquet('MINDsmall_train_processed/news_train_tokenized.parquet')
except FileNotFoundError:
    from google.colab import files
    files.upload()
    news_train_tokenized = pd.read_parquet('news_train_with_embeddings_dense.parquet')

Saving news_train_with_embeddings_dense.parquet to news_train_with_embeddings_dense.parquet


In [ ]:
print(type(news_train_tokenized))
print(news_train_tokenized.shape)
news_train_tokenized.head()

<class 'pandas.core.frame.DataFrame'>
(51282, 19)


news_id   category      subcategory  \
0  N55528  lifestyle  lifestyleroyals   
1  N19639     health       weightloss   
2  N61837       news        newsworld   
3  N53526     health           voices   
4  N38324     health          medical   

                                               title  \
0  The Brands Queen Elizabeth, Prince Charles, an...   
1                      50 Worst Habits For Belly Fat   
2  The Cost of Trump's Aid Freeze in the Trenches...   
3  I Was An NBA Wife. Here's How It Affected My M...   
4  How to Get Rid of Skin Tags, According to a De...   

                                            abstract  \
0  Shop the notebooks, jackets, and more that the...   
1  These seemingly harmless habits are holding yo...   
2  Lt. Ivan Molchanets peeked over a parapet of s...   
3  I felt like I was a fraud, and being an NBA wi...   
4  They seem harmless, but there's a very good re...   

                                             url  \
0  https://assets.msn.com/labs/mind/AAGH0ET.html   
1  https://assets.msn.com/labs/mind/AAB19MK.html   
2  https://assets.msn.com/labs/mind/AAJgNsz.html   
3  https://assets.msn.com/labs/mind/AACk2N6.html   
4  https://assets.msn.com/labs/mind/AAAKEkt.html   

                                      title_entities  \
0  [{"Label": "Prince Philip, Duke of Edinburgh",...   
1  [{"Label": "Adipose tissue", "Type": "C", "Wik...   
2                                                 []   
3                                                 []   
4  [{"Label": "Skin tag", "Type": "C", "WikidataI...   

                                   abstract_entities  \
0                                                 []   
1  [{"Label": "Adipose tissue", "Type": "C", "Wik...   
2  [{"Label": "Ukraine", "Type": "G", "WikidataId...   
3  [{"Label": "National Basketball Association", ...   
4  [{"Label": "Skin tag", "Type": "C", "WikidataI...   

                                                text  \
0  The Brands Queen Elizabeth, Prince Charles, an...   
1  50 Worst Habits For Belly Fat These seemingly ...   
2  The Cost of Trump's Aid Freeze in the Trenches...   
3  I Was An NBA Wife. Here's How It Affected My M...   
4  How to Get Rid of Skin Tags, According to a De...   

                                         text_tokens  \
0  [rand, ueen, lizabeth, rince, harles, rince, h...   
1  [orst, abits, elly, hese, seemingly, harmless,...   
2  [ost, rump, id, reeze, renches, kraines, ar, v...   
3  [n, ife, eres, ow, ffected, ental, ealth, felt...   
4  [ow, et, id, kin, ag, ccording, ermatologist, ...   

                                        title_tokens  \
0  [rand, ueen, lizabeth, rince, harles, rince, h...   
1                                [orst, abits, elly]   
2       [ost, rump, id, reeze, renches, kraines, ar]   
3          [n, ife, eres, ow, ffected, ental, ealth]   
4      [ow, et, id, kin, ag, ccording, ermatologist]   

                                     abstract_tokens  \
0  [hop, notebook, jacket, royal, cant, live, wit...   
1  [hese, seemingly, harmless, habit, holding, ba...   
2  [van, olchanets, peeked, parapet, sand, bag, f...   
3  [felt, like, fraud, wife, didnt, help, n, fact...   
4  [hey, seem, harmless, there, good, reason, sho...   

                                          text_tfidf  \
0  rand ueen lizabeth rince harles rince hilip we...   
1  orst abits elly hese seemingly harmless habit ...   
2  ost rump id reeze renches kraines ar van olcha...   
3  n ife eres ow ffected ental ealth felt like fr...   
4  ow et id kin ag ccording ermatologist hey seem...   

                                            text_w2v  \
0  [rand, ueen, lizabeth, rince, harles, rince, h...   
1  [orst, abits, elly, hese, seemingly, harmless,...   
2  [ost, rump, id, reeze, renches, kraines, ar, v...   
3  [n, ife, eres, ow, ffected, ental, ealth, felt...   
4  [ow, et, id, kin, ag, ccording, ermatologist, ...   

                                           text_bert  \
0  The Brands Qu

In [ ]:
news_train_tokenized['text_w2v'][0] == news_val_tokenized_to_save['text_w2v'][0]

array([ True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True])

In [ ]:
news_train_tokenized['text_w2v'][0]

array(['rand', 'ueen', 'lizabeth', 'rince', 'harles', 'rince', 'hilip',
       'wear', 'hop', 'notebook', 'jacket', 'royal', 'cant', 'live',
       'without'], dtype=object)

In [ ]:
news_val_tokenized_to_save['text_w2v'][0]

array(['rand', 'ueen', 'lizabeth', 'rince', 'harles', 'rince', 'hilip',
       'wear', 'hop', 'notebook', 'jacket', 'royal', 'cant', 'live',
       'without'], dtype=object)

In [ ]:
news_val_tokenized_to_save['abstract'][0]

"Shop the notebooks, jackets, and more that the royals can't live without."

In [ ]:
v_train = news_train_tokenized['vector_w2v'][0]
v_val = news_val_tokenized_to_save['vector_w2v'][0]

np.allclose(v_train, v_val)

False